In [214]:
%reset -sf

In [215]:
# LOAD THIS BEFORE START, run rmdl

#!/usr/bin/env python3
import sys
import math, random
import functools, itertools, collections, heapq, bisect
from collections import Counter, defaultdict, deque

# import numpy as np
import scipy
import matplotlib.pyplot as plt

abc = "abcdefghijklmnopqrstuvwxyz"
abc_map = {c:i for i,c in enumerate(abc)}
M9 = 10**9 + 7  # 998244353
yes, no = "YES", "NO"
d4 = [(1,0),(0,1),(-1,0),(0,-1)]
d8 = [
    (-1,-1),(-1,0),(-1,1),
    (0,-1),(0,0),(0,1),
    (1,-1),(1,0),(1,1)]
d6 = [(2,0),(1,1),(-1,1),(-2,0),(-1,-1),(1,-1)]  # hexagonal layout
MAXINT = sys.maxsize

In [264]:
with open("/Users/htong/Downloads/input.txt") as f:
    arr, brr = f.read().strip().split("\n\n")

In [252]:
arr, brr = """
px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}
""".strip().split("\n\n")

In [265]:
arr = arr.replace("in{", "in12983y1o8uf{")
arr2 = []
for row in arr.split():
    row = row.replace(",", "?", row.count(",") - 1)
    arr2.append(row)
arr = "\n".join(arr2)

In [266]:
print(arr)

mxf{m<2310:phv,zs}
zjr{s>1167:zqt,skq}
hm{x>2975:rkl?x>2818:vhs?a>1062:bkc,kjg}
dkl{m>2321:ctj?x>3530:A,gj}
zk{s<1218:R,A}
zkd{m>864:A?a>488:R,jcp}
zdl{m>3429:A?m>2961:R,R}
xmd{a<576:A?a<1045:R?a<1386:R,A}
kzz{m>1106:R?a<508:R?x>2970:A,R}
gdr{x<139:R?x<183:A?s<222:A,R}
rb{m<3239:R?a<3546:A,A}
xns{x<621:vrz,gm}
nd{s>2927:xnx?x<3488:A?a<434:pg,A}
fl{s<3624:qg?m>952:qf,kjt}
vhm{x<3815:R?s>1774:A?m>643:R,A}
krp{a<2439:R?m>298:R?x<1440:R,R}
njj{x<2856:cjc?x<2936:R?a>598:R,kzz}
jp{m>1745:R?x<1239:A?a<625:lt,qbq}
rhd{a<3515:R?a<3543:A?s<3959:R,R}
htm{a>284:R?m<1961:R?s<2226:R,A}
kjj{a>704:fcn?s<439:lq,R}
jkk{s>1376:R?a<341:A,A}
jt{s<3384:A?x<3143:R,xgd}
bcm{a<1408:R?m<425:R,A}
hc{s>1126:A?m>1159:A,R}
jff{a>3411:A?s<804:jn,A}
knh{s>2643:szq,A}
klj{x>1392:A?a<2628:R,R}
kfx{a>1268:R,A}
cxd{a>1275:A?s>1768:R,R}
clq{a>1293:R?x>3243:R,R}
mpc{a<2362:R,R}
ng{a>578:A?x>1430:A,R}
hh{a>2697:A?s<1336:R?a<2167:A,A}
cjc{a<661:R?s>1113:A?x<2787:A,A}
fgv{m>3553:R?m<3400:A?m>3492:A,A}
pqk{m<110:R,A}
zqt{m<171

In [267]:
global x
global m
global a
global s


In [268]:
vrr = "def " + arr.replace("\n", "\ndef ").replace("{", """():
    global x
    global m
    global a
    global s
    
    if """).replace(":", """: 
        """).replace("?", """()
    elif """).replace(",", """()
    else:
       """).replace("}", "()")

print(vrr)

def mxf(): 
        
    global x
    global m
    global a
    global s
    
    if m<2310: 
        phv()
    else:
       zs()
def zjr(): 
        
    global x
    global m
    global a
    global s
    
    if s>1167: 
        zqt()
    else:
       skq()
def hm(): 
        
    global x
    global m
    global a
    global s
    
    if x>2975: 
        rkl()
    elif x>2818: 
        vhs()
    elif a>1062: 
        bkc()
    else:
       kjg()
def dkl(): 
        
    global x
    global m
    global a
    global s
    
    if m>2321: 
        ctj()
    elif x>3530: 
        A()
    else:
       gj()
def zk(): 
        
    global x
    global m
    global a
    global s
    
    if s<1218: 
        R()
    else:
       A()
def zkd(): 
        
    global x
    global m
    global a
    global s
    
    if m>864: 
        A()
    elif a>488: 
        R()
    else:
       jcp()
def zdl(): 
        
    global x
    global m
    global a
    global s
    
    if m>3429: 
        

In [269]:
def mxf(): 
        
    global x
    global m
    global a
    global s
    
    if m<2310: 
        phv()
    else:
       zs()
def zjr(): 
        
    global x
    global m
    global a
    global s
    
    if s>1167: 
        zqt()
    else:
       skq()
def hm(): 
        
    global x
    global m
    global a
    global s
    
    if x>2975: 
        rkl()
    elif x>2818: 
        vhs()
    elif a>1062: 
        bkc()
    else:
       kjg()
def dkl(): 
        
    global x
    global m
    global a
    global s
    
    if m>2321: 
        ctj()
    elif x>3530: 
        A()
    else:
       gj()
def zk(): 
        
    global x
    global m
    global a
    global s
    
    if s<1218: 
        R()
    else:
       A()
def zkd(): 
        
    global x
    global m
    global a
    global s
    
    if m>864: 
        A()
    elif a>488: 
        R()
    else:
       jcp()
def zdl(): 
        
    global x
    global m
    global a
    global s
    
    if m>3429: 
        A()
    elif m>2961: 
        R()
    else:
       R()
def xmd(): 
        
    global x
    global m
    global a
    global s
    
    if a<576: 
        A()
    elif a<1045: 
        R()
    elif a<1386: 
        R()
    else:
       A()
def kzz(): 
        
    global x
    global m
    global a
    global s
    
    if m>1106: 
        R()
    elif a<508: 
        R()
    elif x>2970: 
        A()
    else:
       R()
def gdr(): 
        
    global x
    global m
    global a
    global s
    
    if x<139: 
        R()
    elif x<183: 
        A()
    elif s<222: 
        A()
    else:
       R()
def rb(): 
        
    global x
    global m
    global a
    global s
    
    if m<3239: 
        R()
    elif a<3546: 
        A()
    else:
       A()
def xns(): 
        
    global x
    global m
    global a
    global s
    
    if x<621: 
        vrz()
    else:
       gm()
def nd(): 
        
    global x
    global m
    global a
    global s
    
    if s>2927: 
        xnx()
    elif x<3488: 
        A()
    elif a<434: 
        pg()
    else:
       A()
def fl(): 
        
    global x
    global m
    global a
    global s
    
    if s<3624: 
        qg()
    elif m>952: 
        qf()
    else:
       kjt()
def vhm(): 
        
    global x
    global m
    global a
    global s
    
    if x<3815: 
        R()
    elif s>1774: 
        A()
    elif m>643: 
        R()
    else:
       A()
def krp(): 
        
    global x
    global m
    global a
    global s
    
    if a<2439: 
        R()
    elif m>298: 
        R()
    elif x<1440: 
        R()
    else:
       R()
def njj(): 
        
    global x
    global m
    global a
    global s
    
    if x<2856: 
        cjc()
    elif x<2936: 
        R()
    elif a>598: 
        R()
    else:
       kzz()
def jp(): 
        
    global x
    global m
    global a
    global s
    
    if m>1745: 
        R()
    elif x<1239: 
        A()
    elif a<625: 
        lt()
    else:
       qbq()
def rhd(): 
        
    global x
    global m
    global a
    global s
    
    if a<3515: 
        R()
    elif a<3543: 
        A()
    elif s<3959: 
        R()
    else:
       R()
def htm(): 
        
    global x
    global m
    global a
    global s
    
    if a>284: 
        R()
    elif m<1961: 
        R()
    elif s<2226: 
        R()
    else:
       A()
def kjj(): 
        
    global x
    global m
    global a
    global s
    
    if a>704: 
        fcn()
    elif s<439: 
        lq()
    else:
       R()
def jkk(): 
        
    global x
    global m
    global a
    global s
    
    if s>1376: 
        R()
    elif a<341: 
        A()
    else:
       A()
def jt(): 
        
    global x
    global m
    global a
    global s
    
    if s<3384: 
        A()
    elif x<3143: 
        R()
    else:
       xgd()
def bcm(): 
        
    global x
    global m
    global a
    global s
    
    if a<1408: 
        R()
    elif m<425: 
        R()
    else:
       A()
def hc(): 
        
    global x
    global m
    global a
    global s
    
    if s>1126: 
        A()
    elif m>1159: 
        A()
    else:
       R()
def jff(): 
        
    global x
    global m
    global a
    global s
    
    if a>3411: 
        A()
    elif s<804: 
        jn()
    else:
       A()
def knh(): 
        
    global x
    global m
    global a
    global s
    
    if s>2643: 
        szq()
    else:
       A()
def klj(): 
        
    global x
    global m
    global a
    global s
    
    if x>1392: 
        A()
    elif a<2628: 
        R()
    else:
       R()
def kfx(): 
        
    global x
    global m
    global a
    global s
    
    if a>1268: 
        R()
    else:
       A()
def cxd(): 
        
    global x
    global m
    global a
    global s
    
    if a>1275: 
        A()
    elif s>1768: 
        R()
    else:
       R()
def clq(): 
        
    global x
    global m
    global a
    global s
    
    if a>1293: 
        R()
    elif x>3243: 
        R()
    else:
       R()
def mpc(): 
        
    global x
    global m
    global a
    global s
    
    if a<2362: 
        R()
    else:
       R()
def ng(): 
        
    global x
    global m
    global a
    global s
    
    if a>578: 
        A()
    elif x>1430: 
        A()
    else:
       R()
def hh(): 
        
    global x
    global m
    global a
    global s
    
    if a>2697: 
        A()
    elif s<1336: 
        R()
    elif a<2167: 
        A()
    else:
       A()
def cjc(): 
        
    global x
    global m
    global a
    global s
    
    if a<661: 
        R()
    elif s>1113: 
        A()
    elif x<2787: 
        A()
    else:
       A()
def fgv(): 
        
    global x
    global m
    global a
    global s
    
    if m>3553: 
        R()
    elif m<3400: 
        A()
    elif m>3492: 
        A()
    else:
       A()
def pqk(): 
        
    global x
    global m
    global a
    global s
    
    if m<110: 
        R()
    else:
       A()
def zqt(): 
        
    global x
    global m
    global a
    global s
    
    if m<1713: 
        mg()
    elif s>1555: 
        jfx()
    elif s<1309: 
        zm()
    else:
       A()
def jrz(): 
        
    global x
    global m
    global a
    global s
    
    if a<665: 
        nh()
    else:
       tgp()
def vsf(): 
        
    global x
    global m
    global a
    global s
    
    if s<2496: 
        R()
    else:
       R()
def xdc(): 
        
    global x
    global m
    global a
    global s
    
    if a>2214: 
        R()
    else:
       A()
def kkm(): 
        
    global x
    global m
    global a
    global s
    
    if s>3217: 
        A()
    elif x<3895: 
        R()
    elif a<857: 
        A()
    else:
       R()
def sbh(): 
        
    global x
    global m
    global a
    global s
    
    if s>1082: 
        ct()
    else:
       sgx()
def kcn(): 
        
    global x
    global m
    global a
    global s
    
    if x<3313: 
        A()
    elif x<3389: 
        mmt()
    else:
       phd()
def sgg(): 
        
    global x
    global m
    global a
    global s
    
    if a<817: 
        rf()
    elif s>2209: 
        nhz()
    else:
       R()
def jd(): 
        
    global x
    global m
    global a
    global s
    
    if s>2746: 
        R()
    elif a>2645: 
        R()
    else:
       R()
def pb(): 
        
    global x
    global m
    global a
    global s
    
    if x>2553: 
        A()
    else:
       qb()
def hq(): 
        
    global x
    global m
    global a
    global s
    
    if a<1200: 
        A()
    elif a<1322: 
        A()
    elif m<968: 
        R()
    else:
       R()
def kfz(): 
        
    global x
    global m
    global a
    global s
    
    if x<3631: 
        A()
    else:
       R()
def fvf(): 
        
    global x
    global m
    global a
    global s
    
    if a>917: 
        drd()
    elif s>3325: 
        qt()
    else:
       kcn()
def ctj(): 
        
    global x
    global m
    global a
    global s
    
    if m>2668: 
        R()
    else:
       A()
def tbl(): 
        
    global x
    global m
    global a
    global s
    
    if x>2400: 
        ql()
    elif m<2122: 
        cs()
    elif m>2504: 
        rq()
    else:
       tpp()
def kn(): 
        
    global x
    global m
    global a
    global s
    
    if m<1806: 
        R()
    elif a<588: 
        R()
    elif x>3133: 
        R()
    else:
       A()
def sv(): 
        
    global x
    global m
    global a
    global s
    
    if a<1325: 
        R()
    elif a>1460: 
        R()
    elif m>1718: 
        R()
    else:
       A()
def rkl(): 
        
    global x
    global m
    global a
    global s
    
    if x>3112: 
        A()
    elif s>3342: 
        R()
    else:
       R()
def mg(): 
        
    global x
    global m
    global a
    global s
    
    if m>1291: 
        R()
    else:
       A()
def tm(): 
        
    global x
    global m
    global a
    global s
    
    if x>1917: 
        bs()
    else:
       A()
def rkq(): 
        
    global x
    global m
    global a
    global s
    
    if a<2465: 
        bxc()
    elif x>1862: 
        nxc()
    else:
       sp()
def tk(): 
        
    global x
    global m
    global a
    global s
    
    if a<1228: 
        A()
    elif a<1455: 
        R()
    else:
       A()
def vts(): 
        
    global x
    global m
    global a
    global s
    
    if s>3466: 
        jq()
    else:
       xns()
def nh(): 
        
    global x
    global m
    global a
    global s
    
    if x<2134: 
        rr()
    elif x<2484: 
        pbv()
    elif m<976: 
        lvp()
    else:
       mv()
def sz(): 
        
    global x
    global m
    global a
    global s
    
    if x<3075: 
        R()
    elif m>359: 
        R()
    else:
       R()
def btr(): 
        
    global x
    global m
    global a
    global s
    
    if x>3197: 
        qkh()
    else:
       ph()
def ff(): 
        
    global x
    global m
    global a
    global s
    
    if m>1946: 
        A()
    elif s<2407: 
        zc()
    else:
       A()
def cmk(): 
        
    global x
    global m
    global a
    global s
    
    if s>2606: 
        R()
    else:
       A()
def jr(): 
        
    global x
    global m
    global a
    global s
    
    if a>804: 
        A()
    elif a<475: 
        R()
    else:
       R()
def rf(): 
        
    global x
    global m
    global a
    global s
    
    if x<415: 
        A()
    elif s>2228: 
        R()
    elif a<735: 
        R()
    else:
       R()
def drd(): 
        
    global x
    global m
    global a
    global s
    
    if s>3365: 
        tk()
    elif s<3276: 
        bcm()
    else:
       A()
def hvq(): 
        
    global x
    global m
    global a
    global s
    
    if s<2401: 
        hsn()
    elif s>3374: 
        fl()
    else:
       xdl()
def vpn(): 
        
    global x
    global m
    global a
    global s
    
    if s<1325: 
        fvm()
    elif m<689: 
        R()
    else:
       R()
def vj(): 
        
    global x
    global m
    global a
    global s
    
    if s<1347: 
        A()
    elif x>673: 
        R()
    else:
       R()
def tnz(): 
        
    global x
    global m
    global a
    global s
    
    if s>999: 
        A()
    elif x>2657: 
        gk()
    else:
       A()
def rft(): 
        
    global x
    global m
    global a
    global s
    
    if a<401: 
        pqp()
    else:
       qdm()
def gkm(): 
        
    global x
    global m
    global a
    global s
    
    if m>430: 
        A()
    elif m>196: 
        A()
    elif s>3374: 
        R()
    else:
       A()
def vhs(): 
        
    global x
    global m
    global a
    global s
    
    if m>623: 
        R()
    else:
       R()
def zzt(): 
        
    global x
    global m
    global a
    global s
    
    if x>3410: 
        A()
    elif m>2991: 
        R()
    elif m<2312: 
        A()
    else:
       A()
def pth(): 
        
    global x
    global m
    global a
    global s
    
    if x<3816: 
        A()
    elif s>3893: 
        A()
    else:
       R()
def vmd(): 
        
    global x
    global m
    global a
    global s
    
    if s>3539: 
        zr()
    else:
       gr()
def dlh(): 
        
    global x
    global m
    global a
    global s
    
    if x<3015: 
        R()
    else:
       A()
def lv(): 
        
    global x
    global m
    global a
    global s
    
    if a<2026: 
        A()
    elif m<292: 
        R()
    elif x>588: 
        R()
    else:
       A()
def pr(): 
        
    global x
    global m
    global a
    global s
    
    if x>3172: 
        rjk()
    elif a<1192: 
        R()
    elif m<595: 
        A()
    else:
       A()
def dzq(): 
        
    global x
    global m
    global a
    global s
    
    if a<812: 
        bgx()
    elif s>1212: 
        nhn()
    else:
       nnd()
def kxf(): 
        
    global x
    global m
    global a
    global s
    
    if x>3058: 
        R()
    else:
       A()
def tgp(): 
        
    global x
    global m
    global a
    global s
    
    if a>1311: 
        kkg()
    elif a>960: 
        jx()
    else:
       hmf()
def npv(): 
        
    global x
    global m
    global a
    global s
    
    if s<668: 
        A()
    elif x>1167: 
        A()
    else:
       R()
def pd(): 
        
    global x
    global m
    global a
    global s
    
    if a<1327: 
        A()
    elif a<1446: 
        rp()
    else:
       R()
def krf(): 
        
    global x
    global m
    global a
    global s
    
    if m>228: 
        A()
    else:
       A()
def cg(): 
        
    global x
    global m
    global a
    global s
    
    if m<3308: 
        nr()
    elif a>1054: 
        fxs()
    else:
       cgf()
def qd(): 
        
    global x
    global m
    global a
    global s
    
    if a>2742: 
        qmb()
    elif x>1719: 
        mt()
    elif s>818: 
        nrs()
    else:
       mpc()
def bsc(): 
        
    global x
    global m
    global a
    global s
    
    if x>1586: 
        R()
    elif a<1207: 
        R()
    else:
       R()
def kcb(): 
        
    global x
    global m
    global a
    global s
    
    if a<1236: 
        A()
    elif s<1368: 
        A()
    else:
       A()
def mf(): 
        
    global x
    global m
    global a
    global s
    
    if a<2852: 
        zv()
    elif m<2664: 
        prx()
    else:
       rb()
def pnj(): 
        
    global x
    global m
    global a
    global s
    
    if a<754: 
        R()
    else:
       rcb()
def zr(): 
        
    global x
    global m
    global a
    global s
    
    if x>1721: 
        A()
    elif x>683: 
        R()
    elif x>265: 
        R()
    else:
       A()
def bn(): 
        
    global x
    global m
    global a
    global s
    
    if s>1397: 
        A()
    elif a<219: 
        R()
    else:
       R()
def hx(): 
        
    global x
    global m
    global a
    global s
    
    if a>953: 
        A()
    elif a>428: 
        A()
    elif x>539: 
        R()
    else:
       R()
def jv(): 
        
    global x
    global m
    global a
    global s
    
    if s<837: 
        pr()
    else:
       fsl()
def qmb(): 
        
    global x
    global m
    global a
    global s
    
    if s<1185: 
        R()
    else:
       A()
def zcf(): 
        
    global x
    global m
    global a
    global s
    
    if x>784: 
        A()
    else:
       R()
def jpf(): 
        
    global x
    global m
    global a
    global s
    
    if s<2409: 
        A()
    elif s<2671: 
        A()
    elif x<1126: 
        A()
    else:
       R()
def kj(): 
        
    global x
    global m
    global a
    global s
    
    if a>836: 
        jv()
    elif x>3467: 
        tjj()
    else:
       rl()
def cnc(): 
        
    global x
    global m
    global a
    global s
    
    if s<3455: 
        A()
    elif s<3662: 
        R()
    elif a>1191: 
        A()
    else:
       R()
def vbg(): 
        
    global x
    global m
    global a
    global s
    
    if s>2501: 
        R()
    elif x<3479: 
        xhj()
    elif x>3748: 
        R()
    else:
       R()
def in12983y1o8uf(): 
        
    global x
    global m
    global a
    global s
    
    if a>1666: 
        mb()
    elif x<1861: 
        dql()
    else:
       sl()
def xg(): 
        
    global x
    global m
    global a
    global s
    
    if x<749: 
        dgs()
    else:
       A()
def pj(): 
        
    global x
    global m
    global a
    global s
    
    if m<830: 
        R()
    else:
       R()
def jb(): 
        
    global x
    global m
    global a
    global s
    
    if m>3287: 
        A()
    elif m>3057: 
        gfh()
    else:
       A()
def zdp(): 
        
    global x
    global m
    global a
    global s
    
    if a>3403: 
        R()
    elif a>3071: 
        R()
    elif m>808: 
        R()
    else:
       A()
def ggc(): 
        
    global x
    global m
    global a
    global s
    
    if x>1433: 
        A()
    elif s>2595: 
        R()
    else:
       R()
def pxr(): 
        
    global x
    global m
    global a
    global s
    
    if s<3463: 
        R()
    elif m<792: 
        R()
    else:
       A()
def lk(): 
        
    global x
    global m
    global a
    global s
    
    if x<724: 
        R()
    elif s<3706: 
        A()
    else:
       trm()
def mjv(): 
        
    global x
    global m
    global a
    global s
    
    if s<3762: 
        A()
    else:
       bxn()
def hl(): 
        
    global x
    global m
    global a
    global s
    
    if a>556: 
        ks()
    elif s<2836: 
        A()
    elif m<1771: 
        kc()
    else:
       kjd()
def gf(): 
        
    global x
    global m
    global a
    global s
    
    if x>1994: 
        R()
    elif m<3457: 
        xkg()
    elif s>3446: 
        R()
    else:
       A()
def bxz(): 
        
    global x
    global m
    global a
    global s
    
    if m<718: 
        R()
    else:
       A()
def bv(): 
        
    global x
    global m
    global a
    global s
    
    if s<2868: 
        ggc()
    else:
       R()
def pg(): 
        
    global x
    global m
    global a
    global s
    
    if a<269: 
        A()
    elif x>3764: 
        R()
    elif m>3066: 
        R()
    else:
       R()
def vxn(): 
        
    global x
    global m
    global a
    global s
    
    if s>1686: 
        R()
    elif a>1312: 
        R()
    elif m<2228: 
        A()
    else:
       A()
def jj(): 
        
    global x
    global m
    global a
    global s
    
    if a>673: 
        A()
    else:
       R()
def zsx(): 
        
    global x
    global m
    global a
    global s
    
    if x>3204: 
        R()
    else:
       A()
def cs(): 
        
    global x
    global m
    global a
    global s
    
    if s<1229: 
        A()
    else:
       R()
def qdm(): 
        
    global x
    global m
    global a
    global s
    
    if a<550: 
        A()
    elif m>833: 
        R()
    else:
       A()
def vkl(): 
        
    global x
    global m
    global a
    global s
    
    if s>365: 
        A()
    elif x<236: 
        gdr()
    elif s<161: 
        zt()
    else:
       vxr()
def cb(): 
        
    global x
    global m
    global a
    global s
    
    if x<1464: 
        A()
    elif x<1513: 
        R()
    elif m<1814: 
        R()
    else:
       R()
def fn(): 
        
    global x
    global m
    global a
    global s
    
    if m>2717: 
        A()
    elif s>3698: 
        A()
    else:
       R()
def jzq(): 
        
    global x
    global m
    global a
    global s
    
    if x>330: 
        A()
    elif m>2865: 
        R()
    else:
       A()
def gn(): 
        
    global x
    global m
    global a
    global s
    
    if a<1001: 
        A()
    else:
       R()
def cvv(): 
        
    global x
    global m
    global a
    global s
    
    if m>817: 
        fr()
    elif x<1974: 
        mrv()
    elif m<316: 
        vd()
    else:
       kh()
def spl(): 
        
    global x
    global m
    global a
    global s
    
    if a<1511: 
        R()
    else:
       A()
def qg(): 
        
    global x
    global m
    global a
    global s
    
    if a<3083: 
        csf()
    elif a>3444: 
        mz()
    else:
       vmd()
def tf(): 
        
    global x
    global m
    global a
    global s
    
    if x>3535: 
        R()
    elif a<669: 
        gxm()
    elif s>2146: 
        clq()
    else:
       A()
def sg(): 
        
    global x
    global m
    global a
    global s
    
    if s<463: 
        lx()
    elif s<936: 
        tq()
    else:
       R()
def pnb(): 
        
    global x
    global m
    global a
    global s
    
    if s<1080: 
        R()
    elif m<3125: 
        A()
    elif m<3446: 
        A()
    else:
       R()
def phd(): 
        
    global x
    global m
    global a
    global s
    
    if x<3427: 
        A()
    else:
       R()
def hmf(): 
        
    global x
    global m
    global a
    global s
    
    if a>804: 
        gsn()
    elif s<1343: 
        btv()
    else:
       vsf()
def cdj(): 
        
    global x
    global m
    global a
    global s
    
    if a>358: 
        mdl()
    elif a<238: 
        A()
    else:
       nk()
def xs(): 
        
    global x
    global m
    global a
    global s
    
    if x>2559: 
        A()
    elif m>1333: 
        A()
    else:
       A()
def phv(): 
        
    global x
    global m
    global a
    global s
    
    if m<2219: 
        R()
    elif m<2257: 
        R()
    elif x<565: 
        A()
    else:
       A()
def rx(): 
        
    global x
    global m
    global a
    global s
    
    if s<2069: 
        nvr()
    elif m>2736: 
        qr()
    else:
       R()
def vld(): 
        
    global x
    global m
    global a
    global s
    
    if x>969: 
        A()
    elif s>1063: 
        A()
    elif m<689: 
        R()
    else:
       A()
def drn(): 
        
    global x
    global m
    global a
    global s
    
    if s>1516: 
        kx()
    elif m<611: 
        kbn()
    else:
       jff()
def jlg(): 
        
    global x
    global m
    global a
    global s
    
    if s<2849: 
        tgr()
    else:
       qlb()
def nrn(): 
        
    global x
    global m
    global a
    global s
    
    if a<1365: 
        xmh()
    elif a>1565: 
        A()
    elif x<1595: 
        R()
    else:
       fgv()
def fx(): 
        
    global x
    global m
    global a
    global s
    
    if x>3184: 
        R()
    elif a<1161: 
        R()
    elif x<2964: 
        A()
    else:
       R()
def nfm(): 
        
    global x
    global m
    global a
    global s
    
    if m<3293: 
        R()
    elif m<3660: 
        A()
    else:
       A()
def cft(): 
        
    global x
    global m
    global a
    global s
    
    if a<735: 
        A()
    elif a>1165: 
        jdq()
    elif m<1013: 
        ctl()
    else:
       xdg()
def grs(): 
        
    global x
    global m
    global a
    global s
    
    if x<630: 
        chb()
    else:
       bp()
def tsq(): 
        
    global x
    global m
    global a
    global s
    
    if s<3424: 
        klj()
    elif s>3632: 
        rkq()
    else:
       tj()
def ll(): 
        
    global x
    global m
    global a
    global s
    
    if a<566: 
        R()
    else:
       R()
def zc(): 
        
    global x
    global m
    global a
    global s
    
    if m<1722: 
        A()
    elif x>1506: 
        A()
    elif x>1464: 
        R()
    else:
       A()
def tmx(): 
        
    global x
    global m
    global a
    global s
    
    if s>3389: 
        A()
    elif x>2770: 
        A()
    else:
       R()
def jdq(): 
        
    global x
    global m
    global a
    global s
    
    if s>1592: 
        A()
    else:
       R()
def hhb(): 
        
    global x
    global m
    global a
    global s
    
    if m<728: 
        R()
    elif s<3217: 
        bf()
    else:
       kfz()
def chb(): 
        
    global x
    global m
    global a
    global s
    
    if s>1613: 
        A()
    else:
       A()
def zcv(): 
        
    global x
    global m
    global a
    global s
    
    if m<1690: 
        A()
    elif a>1254: 
        cb()
    else:
       R()
def gdt(): 
        
    global x
    global m
    global a
    global s
    
    if s<2338: 
        R()
    elif a>529: 
        R()
    else:
       A()
def lrd(): 
        
    global x
    global m
    global a
    global s
    
    if s>1816: 
        A()
    elif s<1737: 
        R()
    elif x>3648: 
        vhm()
    else:
       mxs()
def dl(): 
        
    global x
    global m
    global a
    global s
    
    if s>1645: 
        A()
    else:
       A()
def rdr(): 
        
    global x
    global m
    global a
    global s
    
    if m<3263: 
        lmt()
    else:
       nrn()
def dbp(): 
        
    global x
    global m
    global a
    global s
    
    if x>1326: 
        R()
    else:
       A()
def vr(): 
        
    global x
    global m
    global a
    global s
    
    if m<2215: 
        R()
    else:
       A()
def dqb(): 
        
    global x
    global m
    global a
    global s
    
    if m>1367: 
        A()
    elif s>3854: 
        A()
    else:
       A()
def mj(): 
        
    global x
    global m
    global a
    global s
    
    if x<3291: 
        xmd()
    elif m<1474: 
        A()
    elif x>3594: 
        fp()
    else:
       mk()
def cvq(): 
        
    global x
    global m
    global a
    global s
    
    if s<3167: 
        vz()
    elif x>3484: 
        dzt()
    elif m>1895: 
        qsk()
    else:
       dj()
def hrq(): 
        
    global x
    global m
    global a
    global s
    
    if m>912: 
        zn()
    elif s<2570: 
        pk()
    elif x<3198: 
        A()
    else:
       A()
def tq(): 
        
    global x
    global m
    global a
    global s
    
    if m<525: 
        A()
    else:
       A()
def cx(): 
        
    global x
    global m
    global a
    global s
    
    if s>971: 
        xbb()
    elif a>2286: 
        R()
    elif s>558: 
        lv()
    else:
       R()
def vlz(): 
        
    global x
    global m
    global a
    global s
    
    if s<432: 
        bdx()
    elif m>419: 
        R()
    elif m<201: 
        fh()
    else:
       cm()
def hsm(): 
        
    global x
    global m
    global a
    global s
    
    if x>3894: 
        psb()
    elif s>3500: 
        A()
    else:
       zh()
def pmv(): 
        
    global x
    global m
    global a
    global s
    
    if m>1260: 
        xn()
    elif x<1634: 
        tkk()
    else:
       A()
def mt(): 
        
    global x
    global m
    global a
    global s
    
    if m<3482: 
        lps()
    else:
       R()
def qs(): 
        
    global x
    global m
    global a
    global s
    
    if x>214: 
        R()
    elif m<2255: 
        A()
    else:
       A()
def pf(): 
        
    global x
    global m
    global a
    global s
    
    if m>399: 
        R()
    elif x>3064: 
        R()
    else:
       R()
def fd(): 
        
    global x
    global m
    global a
    global s
    
    if s>3477: 
        A()
    else:
       R()
def cmp(): 
        
    global x
    global m
    global a
    global s
    
    if a>553: 
        A()
    else:
       R()
def gbx(): 
        
    global x
    global m
    global a
    global s
    
    if s>657: 
        R()
    elif m>939: 
        A()
    elif m<824: 
        A()
    else:
       A()
def hsn(): 
        
    global x
    global m
    global a
    global s
    
    if a>2953: 
        drn()
    elif m>656: 
        kzh()
    elif x<1575: 
        xm()
    else:
       vpx()
def vmr(): 
        
    global x
    global m
    global a
    global s
    
    if a<696: 
        A()
    elif m>594: 
        R()
    elif s<2925: 
        A()
    else:
       R()
def hv(): 
        
    global x
    global m
    global a
    global s
    
    if m>2584: 
        R()
    elif m>2556: 
        A()
    else:
       R()
def gq(): 
        
    global x
    global m
    global a
    global s
    
    if m<3154: 
        A()
    elif x>3367: 
        hmd()
    else:
       rsn()
def tpc(): 
        
    global x
    global m
    global a
    global s
    
    if x>3467: 
        A()
    elif m>1865: 
        A()
    elif m<1087: 
        R()
    else:
       R()
def vf(): 
        
    global x
    global m
    global a
    global s
    
    if x>567: 
        A()
    else:
       R()
def xq(): 
        
    global x
    global m
    global a
    global s
    
    if s>2490: 
        ll()
    else:
       jj()
def dx(): 
        
    global x
    global m
    global a
    global s
    
    if m<213: 
        mx()
    elif x<981: 
        qtt()
    elif a<1191: 
        A()
    else:
       ktx()
def sq(): 
        
    global x
    global m
    global a
    global s
    
    if s<518: 
        jtt()
    elif a<1365: 
        fbb()
    else:
       spl()
def ds(): 
        
    global x
    global m
    global a
    global s
    
    if m<1024: 
        R()
    elif x<1332: 
        A()
    else:
       A()
def nn(): 
        
    global x
    global m
    global a
    global s
    
    if m<337: 
        srx()
    else:
       tx()
def xf(): 
        
    global x
    global m
    global a
    global s
    
    if a>645: 
        cnc()
    else:
       R()
def ltv(): 
        
    global x
    global m
    global a
    global s
    
    if m<3188: 
        cp()
    else:
       pt()
def gjp(): 
        
    global x
    global m
    global a
    global s
    
    if a<3444: 
        R()
    else:
       A()
def tbx(): 
        
    global x
    global m
    global a
    global s
    
    if a>2651: 
        bv()
    else:
       zx()
def qf(): 
        
    global x
    global m
    global a
    global s
    
    if a<3156: 
        cmf()
    elif s>3778: 
        gkk()
    elif x<1377: 
        lk()
    else:
       jqq()
def qdf(): 
        
    global x
    global m
    global a
    global s
    
    if m>3031: 
        A()
    elif s>3921: 
        cqp()
    else:
       fg()
def zx(): 
        
    global x
    global m
    global a
    global s
    
    if s>2703: 
        R()
    elif s<2561: 
        gts()
    elif a>2305: 
        A()
    else:
       snk()
def xlj(): 
        
    global x
    global m
    global a
    global s
    
    if a<240: 
        R()
    else:
       R()
def rl(): 
        
    global x
    global m
    global a
    global s
    
    if x>3006: 
        cdj()
    elif a>458: 
        njj()
    else:
       vpn()
def psq(): 
        
    global x
    global m
    global a
    global s
    
    if a>939: 
        tnp()
    elif m<1665: 
        mnk()
    else:
       rpj()
def bc(): 
        
    global x
    global m
    global a
    global s
    
    if a<486: 
        knh()
    elif m<3114: 
        gth()
    else:
       lcr()
def zq(): 
        
    global x
    global m
    global a
    global s
    
    if x<1396: 
        R()
    elif x<1668: 
        A()
    else:
       R()
def xm(): 
        
    global x
    global m
    global a
    global s
    
    if x<973: 
        cx()
    else:
       llb()
def fzx(): 
        
    global x
    global m
    global a
    global s
    
    if x>1946: 
        A()
    elif x>1916: 
        A()
    elif s>2523: 
        dgq()
    else:
       bz()
def gr(): 
        
    global x
    global m
    global a
    global s
    
    if s<3443: 
        A()
    else:
       R()
def rsh(): 
        
    global x
    global m
    global a
    global s
    
    if m<2322: 
        A()
    else:
       R()
def lzn(): 
        
    global x
    global m
    global a
    global s
    
    if m<3606: 
        A()
    elif m<3836: 
        vql()
    elif a<423: 
        R()
    else:
       R()
def rsn(): 
        
    global x
    global m
    global a
    global s
    
    if s<2344: 
        A()
    elif a>1284: 
        A()
    elif m<3610: 
        A()
    else:
       A()
def rg(): 
        
    global x
    global m
    global a
    global s
    
    if m<752: 
        A()
    elif s<3018: 
        A()
    elif s<3072: 
        R()
    else:
       R()
def drs(): 
        
    global x
    global m
    global a
    global s
    
    if a>2000: 
        A()
    else:
       A()
def ctl(): 
        
    global x
    global m
    global a
    global s
    
    if s<1467: 
        R()
    elif x>675: 
        R()
    elif a<885: 
        R()
    else:
       A()
def fvm(): 
        
    global x
    global m
    global a
    global s
    
    if a>168: 
        A()
    elif s>868: 
        R()
    else:
       R()
def mtn(): 
        
    global x
    global m
    global a
    global s
    
    if a>310: 
        R()
    else:
       R()
def rbt(): 
        
    global x
    global m
    global a
    global s
    
    if x>3216: 
        R()
    else:
       A()
def dzt(): 
        
    global x
    global m
    global a
    global s
    
    if m<1618: 
        prn()
    else:
       zxd()
def cp(): 
        
    global x
    global m
    global a
    global s
    
    if x>616: 
        A()
    elif m<2469: 
        qs()
    else:
       R()
def zkk(): 
        
    global x
    global m
    global a
    global s
    
    if a>806: 
        R()
    elif s<1275: 
        A()
    elif a>636: 
        A()
    else:
       R()
def jrp(): 
        
    global x
    global m
    global a
    global s
    
    if a>1285: 
        A()
    else:
       A()
def crv(): 
        
    global x
    global m
    global a
    global s
    
    if m>183: 
        R()
    elif x>1678: 
        mbr()
    else:
       A()
def mk(): 
        
    global x
    global m
    global a
    global s
    
    if m>2781: 
        R()
    elif x<3408: 
        R()
    elif x<3508: 
        A()
    else:
       A()
def qvp(): 
        
    global x
    global m
    global a
    global s
    
    if x<2839: 
        R()
    elif s>2645: 
        A()
    else:
       R()
def tj(): 
        
    global x
    global m
    global a
    global s
    
    if m>3041: 
        jxs()
    elif m<2346: 
        R()
    else:
       A()
def gj(): 
        
    global x
    global m
    global a
    global s
    
    if s<1310: 
        R()
    else:
       R()
def dd(): 
        
    global x
    global m
    global a
    global s
    
    if a>700: 
        dx()
    elif s<1439: 
        xg()
    else:
       rfd()
def ptj(): 
        
    global x
    global m
    global a
    global s
    
    if m>919: 
        cft()
    elif a<819: 
        bq()
    else:
       grs()
def qbq(): 
        
    global x
    global m
    global a
    global s
    
    if a>1005: 
        A()
    elif x>1477: 
        A()
    else:
       A()
def nmh(): 
        
    global x
    global m
    global a
    global s
    
    if a>884: 
        R()
    elif a>308: 
        R()
    elif s<2046: 
        R()
    else:
       hnp()
def lx(): 
        
    global x
    global m
    global a
    global s
    
    if a<901: 
        A()
    elif m>593: 
        R()
    else:
       R()
def pqd(): 
        
    global x
    global m
    global a
    global s
    
    if x>1493: 
        A()
    elif x<1235: 
        A()
    else:
       A()
def bnh(): 
        
    global x
    global m
    global a
    global s
    
    if m<1280: 
        R()
    elif s<3302: 
        R()
    else:
       R()
def tv(): 
        
    global x
    global m
    global a
    global s
    
    if a<332: 
        zg()
    else:
       gdt()
def rnd(): 
        
    global x
    global m
    global a
    global s
    
    if a<1518: 
        R()
    elif a>1596: 
        A()
    else:
       A()
def jlb(): 
        
    global x
    global m
    global a
    global s
    
    if a>2132: 
        R()
    elif s>736: 
        R()
    elif x<1111: 
        A()
    else:
       R()
def mmz(): 
        
    global x
    global m
    global a
    global s
    
    if x>992: 
        R()
    elif a<1403: 
        R()
    elif x<929: 
        A()
    else:
       R()
def qfp(): 
        
    global x
    global m
    global a
    global s
    
    if m>1069: 
        R()
    else:
       A()
def pt(): 
        
    global x
    global m
    global a
    global s
    
    if s>1650: 
        vf()
    elif s>1452: 
        R()
    elif x<446: 
        A()
    else:
       vj()
def tkk(): 
        
    global x
    global m
    global a
    global s
    
    if s<1160: 
        A()
    elif m>1174: 
        R()
    elif a<673: 
        R()
    else:
       A()
def kx(): 
        
    global x
    global m
    global a
    global s
    
    if x>2642: 
        R()
    elif s<1888: 
        A()
    else:
       R()
def dq(): 
        
    global x
    global m
    global a
    global s
    
    if x<1992: 
        sc()
    elif x>2131: 
        mjs()
    else:
       cmp()
def xbb(): 
        
    global x
    global m
    global a
    global s
    
    if s>1608: 
        A()
    elif a<2166: 
        A()
    elif s<1285: 
        R()
    else:
       A()
def btv(): 
        
    global x
    global m
    global a
    global s
    
    if s>646: 
        R()
    else:
       A()
def br(): 
        
    global x
    global m
    global a
    global s
    
    if a<1023: 
        kxf()
    else:
       sds()
def nr(): 
        
    global x
    global m
    global a
    global s
    
    if m>2923: 
        R()
    elif s>3647: 
        R()
    elif m>2756: 
        R()
    else:
       R()
def nhn(): 
        
    global x
    global m
    global a
    global s
    
    if x<1067: 
        ltv()
    else:
       rdr()
def mbr(): 
        
    global x
    global m
    global a
    global s
    
    if x>3214: 
        A()
    else:
       A()
def mrv(): 
        
    global x
    global m
    global a
    global s
    
    if m>400: 
        R()
    elif m<212: 
        A()
    else:
       A()
def vpx(): 
        
    global x
    global m
    global a
    global s
    
    if a>2478: 
        tnz()
    elif s>821: 
        nn()
    elif a<2119: 
        vlz()
    else:
       vkv()
def hn(): 
        
    global x
    global m
    global a
    global s
    
    if x>2956: 
        A()
    else:
       A()
def jz(): 
        
    global x
    global m
    global a
    global s
    
    if a<775: 
        R()
    else:
       A()
def pqp(): 
        
    global x
    global m
    global a
    global s
    
    if m>731: 
        A()
    else:
       A()
def gbf(): 
        
    global x
    global m
    global a
    global s
    
    if a>1519: 
        pvj()
    elif m>715: 
        gbx()
    else:
       rm()
def mz(): 
        
    global x
    global m
    global a
    global s
    
    if m>549: 
        R()
    elif x<2243: 
        krf()
    elif a>3781: 
        R()
    else:
       jm()
def gth(): 
        
    global x
    global m
    global a
    global s
    
    if m>2216: 
        brs()
    elif m>1852: 
        xxk()
    elif a>662: 
        R()
    else:
       A()
def bmn(): 
        
    global x
    global m
    global a
    global s
    
    if s<892: 
        A()
    else:
       R()
def pgh(): 
        
    global x
    global m
    global a
    global s
    
    if s<2509: 
        A()
    elif m<3134: 
        R()
    else:
       R()
def kqd(): 
        
    global x
    global m
    global a
    global s
    
    if s<2377: 
        A()
    else:
       A()
def nhq(): 
        
    global x
    global m
    global a
    global s
    
    if x>538: 
        A()
    elif s<315: 
        R()
    elif m<296: 
        R()
    else:
       R()
def zhr(): 
        
    global x
    global m
    global a
    global s
    
    if m>2625: 
        kqd()
    else:
       tp()
def nrt(): 
        
    global x
    global m
    global a
    global s
    
    if s<1274: 
        A()
    elif x<1313: 
        A()
    else:
       A()
def mp(): 
        
    global x
    global m
    global a
    global s
    
    if a>2286: 
        jd()
    else:
       psr()
def zn(): 
        
    global x
    global m
    global a
    global s
    
    if m>1099: 
        A()
    else:
       R()
def vkv(): 
        
    global x
    global m
    global a
    global s
    
    if m>221: 
        pf()
    elif s<542: 
        qm()
    elif m<110: 
        A()
    else:
       A()
def xdl(): 
        
    global x
    global m
    global a
    global s
    
    if a<2851: 
        nqc()
    elif s<2936: 
        cvv()
    elif x>1808: 
        kp()
    else:
       km()
def kfs(): 
        
    global x
    global m
    global a
    global s
    
    if a>1392: 
        bcv()
    elif s>2659: 
        A()
    else:
       R()
def rcl(): 
        
    global x
    global m
    global a
    global s
    
    if s>3400: 
        R()
    elif m<528: 
        A()
    else:
       R()
def fsl(): 
        
    global x
    global m
    global a
    global s
    
    if s>1571: 
        cxd()
    elif m>988: 
        jrp()
    else:
       A()
def bd(): 
        
    global x
    global m
    global a
    global s
    
    if m<486: 
        R()
    else:
       R()
def sr(): 
        
    global x
    global m
    global a
    global s
    
    if x<445: 
        A()
    else:
       zdl()
def qt(): 
        
    global x
    global m
    global a
    global s
    
    if s<3423: 
        gkm()
    elif x<3311: 
        A()
    elif m>745: 
        R()
    else:
       bqz()
def zfp(): 
        
    global x
    global m
    global a
    global s
    
    if x>1123: 
        nqq()
    else:
       A()
def gbd(): 
        
    global x
    global m
    global a
    global s
    
    if a>1434: 
        A()
    elif x<2274: 
        A()
    elif m>947: 
        R()
    else:
       R()
def kbn(): 
        
    global x
    global m
    global a
    global s
    
    if x<1675: 
        R()
    else:
       sz()
def mb(): 
        
    global x
    global m
    global a
    global s
    
    if m<1446: 
        hvq()
    else:
       ch()
def bcg(): 
        
    global x
    global m
    global a
    global s
    
    if a>2036: 
        A()
    elif a<1912: 
        A()
    else:
       R()
def mv(): 
        
    global x
    global m
    global a
    global s
    
    if s<2113: 
        jkk()
    elif s>2853: 
        fd()
    else:
       xs()
def dgq(): 
        
    global x
    global m
    global a
    global s
    
    if a<1317: 
        R()
    elif m>2464: 
        R()
    elif m<2017: 
        A()
    else:
       A()
def dt(): 
        
    global x
    global m
    global a
    global s
    
    if s<2641: 
        R()
    elif m<702: 
        A()
    else:
       R()
def stf(): 
        
    global x
    global m
    global a
    global s
    
    if m<199: 
        A()
    else:
       R()
def sh(): 
        
    global x
    global m
    global a
    global s
    
    if m>261: 
        R()
    elif a<915: 
        A()
    else:
       R()
def dv(): 
        
    global x
    global m
    global a
    global s
    
    if a>2613: 
        A()
    else:
       R()
def zd(): 
        
    global x
    global m
    global a
    global s
    
    if a<554: 
        cdx()
    elif a<841: 
        kjj()
    elif x<798: 
        mlm()
    else:
       sg()
def jn(): 
        
    global x
    global m
    global a
    global s
    
    if a<3252: 
        A()
    elif x<2123: 
        R()
    else:
       A()
def lvp(): 
        
    global x
    global m
    global a
    global s
    
    if a>277: 
        A()
    else:
       nf()
def fh(): 
        
    global x
    global m
    global a
    global s
    
    if m>123: 
        R()
    else:
       A()
def lt(): 
        
    global x
    global m
    global a
    global s
    
    if x>1632: 
        R()
    elif s<654: 
        R()
    elif a<325: 
        R()
    else:
       R()
def lcr(): 
        
    global x
    global m
    global a
    global s
    
    if a<720: 
        A()
    elif m>3584: 
        A()
    else:
       R()
def jfx(): 
        
    global x
    global m
    global a
    global s
    
    if s>1699: 
        R()
    elif x>616: 
        A()
    else:
       A()
def hg(): 
        
    global x
    global m
    global a
    global s
    
    if s<2617: 
        xmj()
    elif a<850: 
        R()
    elif s>2800: 
        A()
    else:
       A()
def vpd(): 
        
    global x
    global m
    global a
    global s
    
    if m<2602: 
        R()
    elif a<1388: 
        R()
    elif x<2900: 
        rnd()
    else:
       A()
def mc(): 
        
    global x
    global m
    global a
    global s
    
    if x<1595: 
        A()
    elif x<1769: 
        R()
    elif x>1809: 
        A()
    else:
       A()
def hnq(): 
        
    global x
    global m
    global a
    global s
    
    if x>2458: 
        A()
    elif s<1704: 
        R()
    else:
       A()
def bp(): 
        
    global x
    global m
    global a
    global s
    
    if x<1089: 
        A()
    elif x<1479: 
        A()
    else:
       R()
def lps(): 
        
    global x
    global m
    global a
    global s
    
    if x>2602: 
        R()
    else:
       A()
def gmh(): 
        
    global x
    global m
    global a
    global s
    
    if m<2825: 
        R()
    else:
       A()
def dmp(): 
        
    global x
    global m
    global a
    global s
    
    if a<1343: 
        dl()
    elif x<2089: 
        A()
    elif x>2165: 
        A()
    else:
       R()
def bbg(): 
        
    global x
    global m
    global a
    global s
    
    if x<415: 
        A()
    elif m<580: 
        A()
    elif a>1069: 
        mdk()
    else:
       nj()
def qlb(): 
        
    global x
    global m
    global a
    global s
    
    if m<838: 
        nl()
    elif m<1169: 
        pq()
    elif x<799: 
        pl()
    else:
       dxj()
def ms(): 
        
    global x
    global m
    global a
    global s
    
    if s<3815: 
        hn()
    elif a<1068: 
        lqt()
    else:
       qdf()
def lh(): 
        
    global x
    global m
    global a
    global s
    
    if s<885: 
        vkl()
    elif m>1432: 
        pn()
    elif a>1109: 
        pd()
    else:
       vb()
def fg(): 
        
    global x
    global m
    global a
    global s
    
    if s<3875: 
        R()
    elif m<2292: 
        A()
    elif x>2987: 
        A()
    else:
       R()
def sgx(): 
        
    global x
    global m
    global a
    global s
    
    if x>2162: 
        R()
    elif x>2109: 
        A()
    else:
       R()
def fb(): 
        
    global x
    global m
    global a
    global s
    
    if x>2219: 
        bc()
    else:
       sx()
def gh(): 
        
    global x
    global m
    global a
    global s
    
    if a<2425: 
        R()
    else:
       R()
def zj(): 
        
    global x
    global m
    global a
    global s
    
    if s<1096: 
        xlj()
    else:
       R()
def rc(): 
        
    global x
    global m
    global a
    global s
    
    if a>1099: 
        R()
    elif m>1634: 
        jpf()
    elif x<1066: 
        A()
    else:
       R()
def lxp(): 
        
    global x
    global m
    global a
    global s
    
    if a>1410: 
        tpc()
    elif s>2961: 
        qgx()
    else:
       R()
def rsc(): 
        
    global x
    global m
    global a
    global s
    
    if m<134: 
        A()
    elif m>220: 
        A()
    else:
       R()
def xr(): 
        
    global x
    global m
    global a
    global s
    
    if s<789: 
        pj()
    elif m>812: 
        hq()
    elif s<967: 
        xvt()
    else:
       vld()
def jq(): 
        
    global x
    global m
    global a
    global s
    
    if a<892: 
        zcf()
    elif m<3084: 
        fn()
    else:
       R()
def jx(): 
        
    global x
    global m
    global a
    global s
    
    if m<668: 
        bzx()
    elif a>1083: 
        R()
    else:
       A()
def ft(): 
        
    global x
    global m
    global a
    global s
    
    if s>2537: 
        R()
    elif x>3546: 
        R()
    elif x>3014: 
        A()
    else:
       pgh()
def db(): 
        
    global x
    global m
    global a
    global s
    
    if a<998: 
        R()
    elif s<1612: 
        R()
    elif x>2784: 
        A()
    else:
       R()
def vd(): 
        
    global x
    global m
    global a
    global s
    
    if s>2660: 
        pqk()
    elif a<3336: 
        rsc()
    elif a>3760: 
        hvb()
    else:
       A()
def cn(): 
        
    global x
    global m
    global a
    global s
    
    if s>2852: 
        R()
    elif m>2099: 
        rsh()
    else:
       dxk()
def tjz(): 
        
    global x
    global m
    global a
    global s
    
    if x<3743: 
        R()
    elif s>3348: 
        ssv()
    elif s>3252: 
        R()
    else:
       kkm()
def nlt(): 
        
    global x
    global m
    global a
    global s
    
    if a<205: 
        R()
    elif x>1400: 
        R()
    elif a<276: 
        A()
    else:
       A()
def xvt(): 
        
    global x
    global m
    global a
    global s
    
    if m>687: 
        R()
    elif x<799: 
        A()
    else:
       A()
def hp(): 
        
    global x
    global m
    global a
    global s
    
    if m>1850: 
        hg()
    elif m<1700: 
        rc()
    elif s>2663: 
        hl()
    else:
       jr()
def kch(): 
        
    global x
    global m
    global a
    global s
    
    if m>1902: 
        A()
    else:
       R()
def llb(): 
        
    global x
    global m
    global a
    global s
    
    if s<1505: 
        R()
    elif x<1208: 
        R()
    elif x>1337: 
        krp()
    else:
       R()
def tmk(): 
        
    global x
    global m
    global a
    global s
    
    if x<647: 
        tcj()
    elif x<1410: 
        hp()
    elif x>1663: 
        xq()
    else:
       cgj()
def psj(): 
        
    global x
    global m
    global a
    global s
    
    if x<2003: 
        hnh()
    else:
       R()
def szq(): 
        
    global x
    global m
    global a
    global s
    
    if x<2455: 
        A()
    elif x>2571: 
        R()
    else:
       R()
def nxc(): 
        
    global x
    global m
    global a
    global s
    
    if s<3784: 
        A()
    elif s<3926: 
        A()
    elif x>3006: 
        A()
    else:
       A()
def ph(): 
        
    global x
    global m
    global a
    global s
    
    if x>2866: 
        ts()
    else:
       ngq()
def zh(): 
        
    global x
    global m
    global a
    global s
    
    if a>920: 
        A()
    else:
       R()
def md(): 
        
    global x
    global m
    global a
    global s
    
    if x>809: 
        hf()
    elif x<467: 
        lh()
    else:
       zjr()
def kd(): 
        
    global x
    global m
    global a
    global s
    
    if a<1002: 
        jt()
    elif x<3170: 
        vpd()
    else:
       pfg()
def sc(): 
        
    global x
    global m
    global a
    global s
    
    if x<1928: 
        A()
    elif a<406: 
        R()
    elif m>2129: 
        R()
    else:
       A()
def vg(): 
        
    global x
    global m
    global a
    global s
    
    if a<2146: 
        R()
    elif m<798: 
        R()
    elif s>1349: 
        R()
    else:
       A()
def clg(): 
        
    global x
    global m
    global a
    global s
    
    if s<1574: 
        A()
    else:
       A()
def gd(): 
        
    global x
    global m
    global a
    global s
    
    if a<1867: 
        R()
    elif s>1838: 
        R()
    elif m>3359: 
        R()
    else:
       A()
def dxk(): 
        
    global x
    global m
    global a
    global s
    
    if a<463: 
        A()
    elif s<2763: 
        R()
    elif s<2797: 
        R()
    else:
       R()
def prx(): 
        
    global x
    global m
    global a
    global s
    
    if a>3468: 
        vk()
    elif m<2059: 
        rbt()
    elif x>3171: 
        hk()
    else:
       qvp()
def fcn(): 
        
    global x
    global m
    global a
    global s
    
    if x<1040: 
        R()
    elif s>626: 
        R()
    elif x<1572: 
        A()
    else:
       A()
def srx(): 
        
    global x
    global m
    global a
    global s
    
    if m>147: 
        R()
    elif m<80: 
        A()
    else:
       R()
def dzv(): 
        
    global x
    global m
    global a
    global s
    
    if x>2236: 
        zhr()
    elif a<1268: 
        zll()
    else:
       tt()
def bx(): 
        
    global x
    global m
    global a
    global s
    
    if s>2165: 
        A()
    elif x<3424: 
        A()
    elif s<2155: 
        A()
    else:
       R()
def sx(): 
        
    global x
    global m
    global a
    global s
    
    if m<2846: 
        dq()
    elif s<2128: 
        mq()
    elif s>3114: 
        gf()
    else:
       lzn()
def mmt(): 
        
    global x
    global m
    global a
    global s
    
    if m>667: 
        R()
    else:
       R()
def zll(): 
        
    global x
    global m
    global a
    global s
    
    if s>2412: 
        sfz()
    elif x>2017: 
        sbh()
    else:
       tm()
def hnp(): 
        
    global x
    global m
    global a
    global s
    
    if s>2072: 
        R()
    else:
       R()
def pl(): 
        
    global x
    global m
    global a
    global s
    
    if x>413: 
        R()
    elif m<1336: 
        R()
    elif x<211: 
        R()
    else:
       jz()
def dr(): 
        
    global x
    global m
    global a
    global s
    
    if a>1530: 
        R()
    else:
       A()
def qzf(): 
        
    global x
    global m
    global a
    global s
    
    if s>2145: 
        bx()
    elif a<1033: 
        R()
    else:
       kfx()
def rj(): 
        
    global x
    global m
    global a
    global s
    
    if a<3192: 
        A()
    elif s>3193: 
        R()
    elif m>697: 
        A()
    else:
       A()
def fnd(): 
        
    global x
    global m
    global a
    global s
    
    if x>449: 
        A()
    elif s>115: 
        R()
    else:
       R()
def vk(): 
        
    global x
    global m
    global a
    global s
    
    if m<2203: 
        A()
    elif s>2735: 
        R()
    elif x>3249: 
        R()
    else:
       A()
def vxh(): 
        
    global x
    global m
    global a
    global s
    
    if m>559: 
        bxz()
    elif x<777: 
        R()
    else:
       R()
def tpp(): 
        
    global x
    global m
    global a
    global s
    
    if x<1376: 
        hh()
    else:
       bmn()
def nk(): 
        
    global x
    global m
    global a
    global s
    
    if s<810: 
        R()
    elif a>313: 
        A()
    elif a>264: 
        R()
    else:
       A()
def nvr(): 
        
    global x
    global m
    global a
    global s
    
    if s>792: 
        A()
    elif x>1945: 
        R()
    else:
       R()
def zxd(): 
        
    global x
    global m
    global a
    global s
    
    if x<3810: 
        xf()
    elif m<2542: 
        hsm()
    else:
       cg()
def kz(): 
        
    global x
    global m
    global a
    global s
    
    if s<606: 
        R()
    elif m>1077: 
        A()
    else:
       A()
def bkc(): 
        
    global x
    global m
    global a
    global s
    
    if x>2757: 
        A()
    else:
       R()
def xnx(): 
        
    global x
    global m
    global a
    global s
    
    if m>3422: 
        A()
    elif a>711: 
        R()
    elif s<3019: 
        R()
    else:
       A()
def tjf(): 
        
    global x
    global m
    global a
    global s
    
    if a<744: 
        R()
    elif a>1261: 
        A()
    elif a>1039: 
        fx()
    else:
       zsx()
def kc(): 
        
    global x
    global m
    global a
    global s
    
    if m>1735: 
        R()
    else:
       A()
def tgr(): 
        
    global x
    global m
    global a
    global s
    
    if x>786: 
        xjb()
    else:
       bbg()
def td(): 
        
    global x
    global m
    global a
    global s
    
    if a>3522: 
        xqd()
    elif x>2536: 
        R()
    elif x<2142: 
        dlg()
    else:
       rj()
def zrh(): 
        
    global x
    global m
    global a
    global s
    
    if m>1071: 
        md()
    elif s<1184: 
        kg()
    else:
       jrv()
def gsn(): 
        
    global x
    global m
    global a
    global s
    
    if a<885: 
        A()
    elif a>912: 
        R()
    else:
       R()
def ldb(): 
        
    global x
    global m
    global a
    global s
    
    if a>254: 
        R()
    else:
       R()
def qx(): 
        
    global x
    global m
    global a
    global s
    
    if a>982: 
        A()
    elif m<2418: 
        R()
    else:
       R()
def kp(): 
        
    global x
    global m
    global a
    global s
    
    if s<3127: 
        rg()
    elif s>3291: 
        pb()
    elif x<3205: 
        td()
    else:
       hhb()
def rr(): 
        
    global x
    global m
    global a
    global s
    
    if s<2162: 
        R()
    else:
       R()
def sj(): 
        
    global x
    global m
    global a
    global s
    
    if m<572: 
        R()
    else:
       A()
def st(): 
        
    global x
    global m
    global a
    global s
    
    if x<2946: 
        A()
    elif x>3298: 
        A()
    elif a<700: 
        R()
    else:
       R()
def sp(): 
        
    global x
    global m
    global a
    global s
    
    if m<2339: 
        R()
    elif s>3855: 
        A()
    else:
       R()
def zzx(): 
        
    global x
    global m
    global a
    global s
    
    if m<3186: 
        dm()
    else:
       xl()
def bcv(): 
        
    global x
    global m
    global a
    global s
    
    if s<2617: 
        A()
    elif a>1559: 
        R()
    elif x>1488: 
        A()
    else:
       A()
def xhj(): 
        
    global x
    global m
    global a
    global s
    
    if s>2477: 
        A()
    else:
       R()
def fp(): 
        
    global x
    global m
    global a
    global s
    
    if x>3859: 
        R()
    else:
       R()
def nx(): 
        
    global x
    global m
    global a
    global s
    
    if m<1903: 
        A()
    else:
       bsc()
def gpf(): 
        
    global x
    global m
    global a
    global s
    
    if m<821: 
        A()
    elif m>1201: 
        R()
    elif s<3661: 
        R()
    else:
       R()
def bxc(): 
        
    global x
    global m
    global a
    global s
    
    if x>1591: 
        R()
    else:
       A()
def mx(): 
        
    global x
    global m
    global a
    global s
    
    if m>85: 
        R()
    else:
       R()
def dql(): 
        
    global x
    global m
    global a
    global s
    
    if s>1910: 
        lpf()
    elif m>2088: 
        dzq()
    else:
       zrh()
def fvx(): 
        
    global x
    global m
    global a
    global s
    
    if a<1533: 
        A()
    else:
       A()
def snb(): 
        
    global x
    global m
    global a
    global s
    
    if m>1043: 
        A()
    elif s<1569: 
        vg()
    elif m<829: 
        A()
    else:
       mh()
def rp(): 
        
    global x
    global m
    global a
    global s
    
    if x<289: 
        A()
    else:
       A()
def xxk(): 
        
    global x
    global m
    global a
    global s
    
    if m<1974: 
        A()
    elif s<1985: 
        A()
    else:
       R()
def qr(): 
        
    global x
    global m
    global a
    global s
    
    if s<3186: 
        A()
    elif a>1382: 
        R()
    elif s>3578: 
        A()
    else:
       A()
def jrv(): 
        
    global x
    global m
    global a
    global s
    
    if m<380: 
        dd()
    elif m<830: 
        hhr()
    else:
       ptj()
def skq(): 
        
    global x
    global m
    global a
    global s
    
    if x>635: 
        A()
    else:
       hx()
def rpj(): 
        
    global x
    global m
    global a
    global s
    
    if s>3338: 
        kn()
    elif a>391: 
        A()
    else:
       R()
def zs(): 
        
    global x
    global m
    global a
    global s
    
    if m>2355: 
        R()
    elif a>863: 
        A()
    elif s<3650: 
        A()
    else:
       A()
def rvd(): 
        
    global x
    global m
    global a
    global s
    
    if s>1220: 
        R()
    elif s<494: 
        jzq()
    elif s<762: 
        nlj()
    else:
       A()
def qm(): 
        
    global x
    global m
    global a
    global s
    
    if x<2515: 
        R()
    else:
       R()
def sqt(): 
        
    global x
    global m
    global a
    global s
    
    if a<1023: 
        A()
    else:
       R()
def lcl(): 
        
    global x
    global m
    global a
    global s
    
    if m<2710: 
        A()
    elif m<3464: 
        R()
    elif m<3759: 
        R()
    else:
       R()
def gkk(): 
        
    global x
    global m
    global a
    global s
    
    if s<3899: 
        R()
    elif a>3581: 
        A()
    elif a<3426: 
        R()
    else:
       rhd()
def fr(): 
        
    global x
    global m
    global a
    global s
    
    if s<2683: 
        njl()
    else:
       R()
def nhz(): 
        
    global x
    global m
    global a
    global s
    
    if a<947: 
        A()
    elif m>1908: 
        R()
    elif m>1673: 
        A()
    else:
       A()
def rlp(): 
        
    global x
    global m
    global a
    global s
    
    if s<774: 
        A()
    else:
       R()
def bf(): 
        
    global x
    global m
    global a
    global s
    
    if s<3175: 
        A()
    else:
       R()
def psr(): 
        
    global x
    global m
    global a
    global s
    
    if x>1061: 
        R()
    elif a<1875: 
        A()
    elif x>357: 
        R()
    else:
       A()
def nqs(): 
        
    global x
    global m
    global a
    global s
    
    if x<319: 
        A()
    elif s<3699: 
        A()
    elif x>463: 
        kch()
    else:
       R()
def fxs(): 
        
    global x
    global m
    global a
    global s
    
    if s<3617: 
        R()
    elif a<1444: 
        A()
    else:
       A()
def dxj(): 
        
    global x
    global m
    global a
    global s
    
    if m>1297: 
        gn()
    else:
       flt()
def mhl(): 
        
    global x
    global m
    global a
    global s
    
    if m>1106: 
        R()
    elif s>1412: 
        A()
    elif s<1288: 
        R()
    else:
       R()
def cgf(): 
        
    global x
    global m
    global a
    global s
    
    if x<3874: 
        R()
    elif m>3565: 
        A()
    else:
       A()
def qtt(): 
        
    global x
    global m
    global a
    global s
    
    if x<603: 
        A()
    elif m<300: 
        R()
    elif x>743: 
        R()
    else:
       R()
def sd(): 
        
    global x
    global m
    global a
    global s
    
    if x<3134: 
        A()
    elif x>3645: 
        qfp()
    elif x<3407: 
        hc()
    else:
       R()
def lpf(): 
        
    global x
    global m
    global a
    global s
    
    if m>2444: 
        cz()
    elif m<1547: 
        jlg()
    elif s<3133: 
        tmk()
    else:
       bt()
def bbp(): 
        
    global x
    global m
    global a
    global s
    
    if a>1097: 
        lxp()
    elif m<2430: 
        mvh()
    else:
       nd()
def kkg(): 
        
    global x
    global m
    global a
    global s
    
    if s>1504: 
        fvx()
    elif s>961: 
        gbd()
    elif x<2356: 
        dr()
    else:
       A()
def vxr(): 
        
    global x
    global m
    global a
    global s
    
    if m<1531: 
        A()
    elif m<1719: 
        R()
    elif x<357: 
        R()
    else:
       R()
def mvh(): 
        
    global x
    global m
    global a
    global s
    
    if m<1146: 
        vmr()
    elif a<380: 
        R()
    else:
       R()
def csf(): 
        
    global x
    global m
    global a
    global s
    
    if x>2650: 
        pxr()
    elif a<2576: 
        R()
    else:
       vv()
def qzs(): 
        
    global x
    global m
    global a
    global s
    
    if m<868: 
        R()
    elif x<471: 
        A()
    else:
       R()
def sl(): 
        
    global x
    global m
    global a
    global s
    
    if x<2677: 
        hrh()
    elif s>1997: 
        cvq()
    else:
       xpq()
def hnh(): 
        
    global x
    global m
    global a
    global s
    
    if s<2435: 
        A()
    elif a<1516: 
        A()
    elif a>1606: 
        R()
    else:
       R()
def bq(): 
        
    global x
    global m
    global a
    global s
    
    if x<943: 
        qzs()
    elif m>870: 
        cr()
    else:
       mvd()
def rq(): 
        
    global x
    global m
    global a
    global s
    
    if a>2886: 
        nrt()
    elif m>2662: 
        gh()
    else:
       hv()
def dm(): 
        
    global x
    global m
    global a
    global s
    
    if a<495: 
        mtn()
    elif a>678: 
        bpt()
    else:
       ng()
def mxs(): 
        
    global x
    global m
    global a
    global s
    
    if a>470: 
        R()
    elif x>3575: 
        A()
    else:
       A()
def gxm(): 
        
    global x
    global m
    global a
    global s
    
    if m>1455: 
        A()
    elif a>431: 
        R()
    else:
       R()
def bzx(): 
        
    global x
    global m
    global a
    global s
    
    if a>1117: 
        R()
    elif s<2331: 
        A()
    else:
       A()
def tcj(): 
        
    global x
    global m
    global a
    global s
    
    if a>1108: 
        jkx()
    elif s>2661: 
        cn()
    elif a>647: 
        sgg()
    else:
       htm()
def zl(): 
        
    global x
    global m
    global a
    global s
    
    if x<1326: 
        R()
    elif a<186: 
        R()
    else:
       A()
def bqz(): 
        
    global x
    global m
    global a
    global s
    
    if a>544: 
        A()
    elif a<307: 
        R()
    else:
       A()
def xkg(): 
        
    global x
    global m
    global a
    global s
    
    if x<1915: 
        A()
    else:
       A()
def cdx(): 
        
    global x
    global m
    global a
    global s
    
    if a>339: 
        rlp()
    elif x<772: 
        A()
    elif a>152: 
        npv()
    else:
       jk()
def bz(): 
        
    global x
    global m
    global a
    global s
    
    if a>1306: 
        R()
    elif x<1884: 
        R()
    elif x<1901: 
        R()
    else:
       A()
def hd(): 
        
    global x
    global m
    global a
    global s
    
    if x>441: 
        A()
    elif m>802: 
        A()
    else:
       A()
def kh(): 
        
    global x
    global m
    global a
    global s
    
    if x>3037: 
        R()
    else:
       cmk()
def mn(): 
        
    global x
    global m
    global a
    global s
    
    if a<482: 
        A()
    else:
       A()
def pk(): 
        
    global x
    global m
    global a
    global s
    
    if s<2557: 
        A()
    else:
       R()
def nqq(): 
        
    global x
    global m
    global a
    global s
    
    if a>559: 
        A()
    elif s<3600: 
        A()
    elif x>1458: 
        A()
    else:
       R()
def nrs(): 
        
    global x
    global m
    global a
    global s
    
    if a>2275: 
        R()
    elif a>2016: 
        R()
    else:
       gd()
def rpk(): 
        
    global x
    global m
    global a
    global s
    
    if s<2091: 
        nmh()
    elif s>2183: 
        mj()
    elif m<2279: 
        tf()
    else:
       qzf()
def mlm(): 
        
    global x
    global m
    global a
    global s
    
    if x<388: 
        R()
    elif m>571: 
        A()
    elif s<752: 
        nhq()
    else:
       sh()
def jxs(): 
        
    global x
    global m
    global a
    global s
    
    if a>2736: 
        A()
    else:
       R()
def ks(): 
        
    global x
    global m
    global a
    global s
    
    if x<974: 
        A()
    elif a>1269: 
        A()
    elif x>1159: 
        R()
    else:
       R()
def xmh(): 
        
    global x
    global m
    global a
    global s
    
    if m<3623: 
        A()
    elif m<3765: 
        A()
    else:
       R()
def hrh(): 
        
    global x
    global m
    global a
    global s
    
    if m<1654: 
        jrz()
    elif a<980: 
        fb()
    else:
       dzv()
def tp(): 
        
    global x
    global m
    global a
    global s
    
    if m<2079: 
        hnq()
    else:
       kcb()
def fv(): 
        
    global x
    global m
    global a
    global s
    
    if m>1406: 
        ft()
    elif s<2551: 
        vbg()
    else:
       hrq()
def bpt(): 
        
    global x
    global m
    global a
    global s
    
    if a<758: 
        A()
    elif m<2540: 
        A()
    else:
       R()
def vql(): 
        
    global x
    global m
    global a
    global s
    
    if x<1984: 
        A()
    elif a>515: 
        A()
    elif a<192: 
        A()
    else:
       A()
def bs(): 
        
    global x
    global m
    global a
    global s
    
    if m>2823: 
        A()
    else:
       A()
def nf(): 
        
    global x
    global m
    global a
    global s
    
    if m>644: 
        R()
    else:
       R()
def ssv(): 
        
    global x
    global m
    global a
    global s
    
    if s<3440: 
        A()
    else:
       A()
def rfd(): 
        
    global x
    global m
    global a
    global s
    
    if s<1718: 
        clg()
    elif s>1797: 
        stf()
    elif m<251: 
        R()
    else:
       A()
def lq(): 
        
    global x
    global m
    global a
    global s
    
    if a>646: 
        R()
    elif x>1191: 
        R()
    else:
       R()
def rv(): 
        
    global x
    global m
    global a
    global s
    
    if a<531: 
        zj()
    else:
       rvd()
def tz(): 
        
    global x
    global m
    global a
    global s
    
    if s>403: 
        A()
    elif s>222: 
        R()
    else:
       A()
def gk(): 
        
    global x
    global m
    global a
    global s
    
    if a<2728: 
        R()
    elif x<3291: 
        A()
    elif a>2857: 
        R()
    else:
       R()
def vjj(): 
        
    global x
    global m
    global a
    global s
    
    if x>745: 
        zfp()
    else:
       nqs()
def cqp(): 
        
    global x
    global m
    global a
    global s
    
    if x>2993: 
        A()
    elif m<2484: 
        R()
    elif a<1361: 
        A()
    else:
       A()
def xqd(): 
        
    global x
    global m
    global a
    global s
    
    if s>3201: 
        A()
    else:
       R()
def cr(): 
        
    global x
    global m
    global a
    global s
    
    if s>1438: 
        A()
    elif a<283: 
        A()
    elif x<1533: 
        A()
    else:
       A()
def xdg(): 
        
    global x
    global m
    global a
    global s
    
    if s>1494: 
        A()
    else:
       R()
def xjb(): 
        
    global x
    global m
    global a
    global s
    
    if s<2508: 
        pqd()
    elif s>2732: 
        xcm()
    elif s>2589: 
        dt()
    else:
       ds()
def nnd(): 
        
    global x
    global m
    global a
    global s
    
    if a>1189: 
        sq()
    else:
       dck()
def brs(): 
        
    global x
    global m
    global a
    global s
    
    if s<1860: 
        A()
    elif x>2522: 
        R()
    else:
       A()
def jm(): 
        
    global x
    global m
    global a
    global s
    
    if s<3472: 
        A()
    elif m>227: 
        A()
    elif m<99: 
        A()
    else:
       A()
def mh(): 
        
    global x
    global m
    global a
    global s
    
    if m<948: 
        A()
    elif m>1009: 
        R()
    else:
       R()
def lm(): 
        
    global x
    global m
    global a
    global s
    
    if s<2631: 
        A()
    else:
       A()
def qgx(): 
        
    global x
    global m
    global a
    global s
    
    if m>1733: 
        A()
    elif a<1279: 
        A()
    elif s>3061: 
        R()
    else:
       A()
def kjg(): 
        
    global x
    global m
    global a
    global s
    
    if m>757: 
        tmx()
    else:
       A()
def njl(): 
        
    global x
    global m
    global a
    global s
    
    if a<3412: 
        R()
    elif s>2568: 
        R()
    elif s>2458: 
        R()
    else:
       A()
def mnk(): 
        
    global x
    global m
    global a
    global s
    
    if m>1403: 
        phx()
    elif m>1317: 
        R()
    elif a>417: 
        st()
    else:
       bnh()
def vb(): 
        
    global x
    global m
    global a
    global s
    
    if a>478: 
        zkk()
    else:
       bn()
def tnp(): 
        
    global x
    global m
    global a
    global s
    
    if x<3103: 
        A()
    elif x>3353: 
        dkv()
    elif m<1534: 
        A()
    else:
       sv()
def plz(): 
        
    global x
    global m
    global a
    global s
    
    if a<2245: 
        A()
    else:
       A()
def hhr(): 
        
    global x
    global m
    global a
    global s
    
    if s>1496: 
        vxh()
    else:
       sj()
def nv(): 
        
    global x
    global m
    global a
    global s
    
    if m>3730: 
        R()
    else:
       R()
def km(): 
        
    global x
    global m
    global a
    global s
    
    if x<978: 
        hd()
    else:
       rqf()
def dgs(): 
        
    global x
    global m
    global a
    global s
    
    if m<245: 
        A()
    elif s<1274: 
        A()
    else:
       R()
def tnb(): 
        
    global x
    global m
    global a
    global s
    
    if s>3817: 
        pth()
    elif x>3774: 
        A()
    else:
       gpf()
def cq(): 
        
    global x
    global m
    global a
    global s
    
    if a<1034: 
        mn()
    else:
       mmz()
def dck(): 
        
    global x
    global m
    global a
    global s
    
    if s>410: 
        cxs()
    elif m<2779: 
        qx()
    else:
       jb()
def cm(): 
        
    global x
    global m
    global a
    global s
    
    if a>1896: 
        R()
    else:
       A()
def kms(): 
        
    global x
    global m
    global a
    global s
    
    if a>2292: 
        R()
    elif x<1613: 
        drs()
    elif m>1261: 
        bcg()
    else:
       A()
def bdx(): 
        
    global x
    global m
    global a
    global s
    
    if m>310: 
        R()
    elif s<240: 
        A()
    elif s>356: 
        R()
    else:
       A()
def rqf(): 
        
    global x
    global m
    global a
    global s
    
    if s>3178: 
        A()
    else:
       zdp()
def xmj(): 
        
    global x
    global m
    global a
    global s
    
    if m>2206: 
        A()
    else:
       R()
def hf(): 
        
    global x
    global m
    global a
    global s
    
    if m>1482: 
        jp()
    elif x<1267: 
        cq()
    else:
       pmv()
def vv(): 
        
    global x
    global m
    global a
    global s
    
    if s<3498: 
        A()
    elif x>954: 
        A()
    elif x>375: 
        A()
    else:
       R()
def bxn(): 
        
    global x
    global m
    global a
    global s
    
    if m>622: 
        A()
    elif x<3163: 
        A()
    else:
       A()
def lqt(): 
        
    global x
    global m
    global a
    global s
    
    if m<3266: 
        A()
    elif a<374: 
        A()
    elif a<656: 
        R()
    else:
       A()
def kjd(): 
        
    global x
    global m
    global a
    global s
    
    if a>215: 
        R()
    else:
       A()
def lvs(): 
        
    global x
    global m
    global a
    global s
    
    if s<2888: 
        lm()
    else:
       A()
def jcz(): 
        
    global x
    global m
    global a
    global s
    
    if x>988: 
        xd()
    else:
       mxf()
def bt(): 
        
    global x
    global m
    global a
    global s
    
    if m<2092: 
        vjj()
    else:
       jcz()
def gm(): 
        
    global x
    global m
    global a
    global s
    
    if a>795: 
        A()
    elif m>3022: 
        R()
    else:
       R()
def prn(): 
        
    global x
    global m
    global a
    global s
    
    if s>3549: 
        tnb()
    else:
       tjz()
def tjj(): 
        
    global x
    global m
    global a
    global s
    
    if s<1166: 
        zkd()
    elif s>1652: 
        lrd()
    elif x<3663: 
        sxt()
    else:
       rft()
def tt(): 
        
    global x
    global m
    global a
    global s
    
    if a>1417: 
        psj()
    elif x>2002: 
        dmp()
    elif a>1352: 
        rx()
    else:
       fzx()
def nj(): 
        
    global x
    global m
    global a
    global s
    
    if m<1121: 
        R()
    else:
       R()
def qkh(): 
        
    global x
    global m
    global a
    global s
    
    if m>3145: 
        pnj()
    else:
       dkl()
def ch(): 
        
    global x
    global m
    global a
    global s
    
    if s>2396: 
        dg()
    else:
       cv()
def zm(): 
        
    global x
    global m
    global a
    global s
    
    if x>625: 
        A()
    elif x<542: 
        A()
    elif m>1927: 
        A()
    else:
       R()
def lmt(): 
        
    global x
    global m
    global a
    global s
    
    if m>2506: 
        R()
    elif s<1559: 
        R()
    elif m>2329: 
        mc()
    else:
       vxn()
def fbb(): 
        
    global x
    global m
    global a
    global s
    
    if a<1249: 
        R()
    elif s<765: 
        A()
    else:
       R()
def nlj(): 
        
    global x
    global m
    global a
    global s
    
    if s<671: 
        R()
    elif x<425: 
        R()
    else:
       R()
def frt(): 
        
    global x
    global m
    global a
    global s
    
    if m<454: 
        srz()
    elif a>1436: 
        gbf()
    else:
       xr()
def cv(): 
        
    global x
    global m
    global a
    global s
    
    if m>2927: 
        qd()
    else:
       tbl()
def vz(): 
        
    global x
    global m
    global a
    global s
    
    if s>2596: 
        bbp()
    elif s<2247: 
        rpk()
    elif s>2462: 
        fv()
    else:
       ssd()
def pbv(): 
        
    global x
    global m
    global a
    global s
    
    if s>1407: 
        ldb()
    elif s>929: 
        A()
    else:
       kz()
def bgx(): 
        
    global x
    global m
    global a
    global s
    
    if x<941: 
        rv()
    else:
       zzx()
def pfg(): 
        
    global x
    global m
    global a
    global s
    
    if x>3349: 
        zzt()
    else:
       nfm()
def ssd(): 
        
    global x
    global m
    global a
    global s
    
    if m<2231: 
        tjf()
    elif a>683: 
        gq()
    else:
       tv()
def cmf(): 
        
    global x
    global m
    global a
    global s
    
    if s<3771: 
        xj()
    elif m<1213: 
        plz()
    else:
       dqb()
def qsk(): 
        
    global x
    global m
    global a
    global s
    
    if s<3510: 
        kd()
    else:
       ms()
def mjs(): 
        
    global x
    global m
    global a
    global s
    
    if s<1429: 
        A()
    else:
       A()
def dj(): 
        
    global x
    global m
    global a
    global s
    
    if s>3501: 
        br()
    elif m>1254: 
        psq()
    elif x<3181: 
        hm()
    else:
       fvf()
def hmd(): 
        
    global x
    global m
    global a
    global s
    
    if m<3500: 
        R()
    elif m>3704: 
        R()
    else:
       A()
def tx(): 
        
    global x
    global m
    global a
    global s
    
    if x>2397: 
        R()
    elif x<1870: 
        A()
    elif s<1560: 
        A()
    else:
       R()
def ts(): 
        
    global x
    global m
    global a
    global s
    
    if a<1021: 
        A()
    elif x>2977: 
        R()
    else:
       lcl()
def sfz(): 
        
    global x
    global m
    global a
    global s
    
    if m>2706: 
        A()
    elif s>3253: 
        A()
    else:
       A()
def vrz(): 
        
    global x
    global m
    global a
    global s
    
    if m>3406: 
        R()
    elif m<2945: 
        A()
    else:
       A()
def psb(): 
        
    global x
    global m
    global a
    global s
    
    if x>3943: 
        R()
    else:
       R()
def hvb(): 
        
    global x
    global m
    global a
    global s
    
    if m>189: 
        A()
    elif m<82: 
        R()
    else:
       A()
def xk(): 
        
    global x
    global m
    global a
    global s
    
    if x<1541: 
        dbp()
    elif m<3333: 
        xpg()
    elif s<2457: 
        R()
    else:
       nv()
def jkx(): 
        
    global x
    global m
    global a
    global s
    
    if s<2566: 
        R()
    else:
       A()
def trm(): 
        
    global x
    global m
    global a
    global s
    
    if a>3458: 
        A()
    else:
       A()
def xl(): 
        
    global x
    global m
    global a
    global s
    
    if a>319: 
        A()
    elif m>3484: 
        zl()
    else:
       nlt()
def mvd(): 
        
    global x
    global m
    global a
    global s
    
    if a>391: 
        A()
    elif m>852: 
        R()
    else:
       A()
def mdk(): 
        
    global x
    global m
    global a
    global s
    
    if m<1132: 
        R()
    elif m>1384: 
        A()
    else:
       R()
def ql(): 
        
    global x
    global m
    global a
    global s
    
    if a>2685: 
        dlh()
    else:
       xdc()
def tg(): 
        
    global x
    global m
    global a
    global s
    
    if x>1137: 
        xk()
    else:
       sr()
def xcm(): 
        
    global x
    global m
    global a
    global s
    
    if s>2808: 
        A()
    else:
       R()
def dkv(): 
        
    global x
    global m
    global a
    global s
    
    if m>1597: 
        A()
    elif a<1295: 
        R()
    elif m>1451: 
        A()
    else:
       A()
def pq(): 
        
    global x
    global m
    global a
    global s
    
    if x>685: 
        zq()
    else:
       A()
def hcq(): 
        
    global x
    global m
    global a
    global s
    
    if m<1264: 
        A()
    elif a>3456: 
        R()
    elif x<3097: 
        R()
    else:
       R()
def gfh(): 
        
    global x
    global m
    global a
    global s
    
    if a<942: 
        R()
    elif s>187: 
        A()
    elif m>3151: 
        A()
    else:
       R()
def nqc(): 
        
    global x
    global m
    global a
    global s
    
    if m>952: 
        kms()
    elif m<498: 
        crv()
    elif x<1886: 
        mp()
    else:
       lvs()
def zv(): 
        
    global x
    global m
    global a
    global s
    
    if a<2363: 
        A()
    elif s>2848: 
        A()
    else:
       dv()
def kg(): 
        
    global x
    global m
    global a
    global s
    
    if a<990: 
        zd()
    else:
       frt()
def sxt(): 
        
    global x
    global m
    global a
    global s
    
    if x>3571: 
        mhl()
    elif s>1399: 
        R()
    else:
       R()
def rcb(): 
        
    global x
    global m
    global a
    global s
    
    if s>831: 
        R()
    else:
       R()
def ct(): 
        
    global x
    global m
    global a
    global s
    
    if m<2782: 
        R()
    elif x<2144: 
        R()
    else:
       R()
def jk(): 
        
    global x
    global m
    global a
    global s
    
    if x>1280: 
        R()
    elif a<62: 
        R()
    elif a<97: 
        A()
    else:
       A()
def xpg(): 
        
    global x
    global m
    global a
    global s
    
    if s<2389: 
        R()
    elif x>1653: 
        R()
    elif x>1605: 
        A()
    else:
       A()
def rm(): 
        
    global x
    global m
    global a
    global s
    
    if s>756: 
        R()
    elif s<272: 
        A()
    else:
       A()
def zt(): 
        
    global x
    global m
    global a
    global s
    
    if m<1571: 
        R()
    elif x>357: 
        R()
    elif s<90: 
        R()
    else:
       R()
def kzh(): 
        
    global x
    global m
    global a
    global s
    
    if x>2664: 
        sd()
    elif s<1123: 
        jlb()
    else:
       snb()
def hk(): 
        
    global x
    global m
    global a
    global s
    
    if a>3113: 
        A()
    elif m<2299: 
        R()
    else:
       R()
def xgd(): 
        
    global x
    global m
    global a
    global s
    
    if s>3436: 
        A()
    elif s<3413: 
        A()
    elif m<2980: 
        A()
    else:
       R()
def xn(): 
        
    global x
    global m
    global a
    global s
    
    if a<733: 
        A()
    elif x>1650: 
        R()
    elif a<1166: 
        A()
    else:
       R()
def zg(): 
        
    global x
    global m
    global a
    global s
    
    if a>211: 
        R()
    else:
       R()
def phx(): 
        
    global x
    global m
    global a
    global s
    
    if m>1520: 
        R()
    elif s<3316: 
        R()
    else:
       A()
def nl(): 
        
    global x
    global m
    global a
    global s
    
    if x>1066: 
        rcl()
    else:
       R()
def gts(): 
        
    global x
    global m
    global a
    global s
    
    if a<2156: 
        R()
    else:
       A()
def cz(): 
        
    global x
    global m
    global a
    global s
    
    if s<3196: 
        tg()
    else:
       vts()
def flt(): 
        
    global x
    global m
    global a
    global s
    
    if x<1306: 
        R()
    elif m>1253: 
        A()
    elif s<3466: 
        R()
    else:
       R()
def mq(): 
        
    global x
    global m
    global a
    global s
    
    if m<3476: 
        R()
    elif a>643: 
        R()
    elif m>3779: 
        A()
    else:
       zk()
def xpq(): 
        
    global x
    global m
    global a
    global s
    
    if m>1741: 
        btr()
    else:
       kj()
def dlg(): 
        
    global x
    global m
    global a
    global s
    
    if s<3232: 
        R()
    elif s>3262: 
        R()
    else:
       R()
def jcp(): 
        
    global x
    global m
    global a
    global s
    
    if m<292: 
        A()
    elif x<3654: 
        A()
    elif a<245: 
        A()
    else:
       A()
def sds(): 
        
    global x
    global m
    global a
    global s
    
    if a>1440: 
        A()
    elif x<3037: 
        R()
    else:
       A()
def kjt(): 
        
    global x
    global m
    global a
    global s
    
    if x<2493: 
        bd()
    else:
       mjv()
def xd(): 
        
    global x
    global m
    global a
    global s
    
    if x>1460: 
        R()
    else:
       vr()
def dg(): 
        
    global x
    global m
    global a
    global s
    
    if s>3110: 
        tsq()
    elif x>2350: 
        mf()
    else:
       tbx()
def snk(): 
        
    global x
    global m
    global a
    global s
    
    if a>2064: 
        A()
    elif m>2299: 
        R()
    else:
       A()
def xj(): 
        
    global x
    global m
    global a
    global s
    
    if s<3720: 
        R()
    elif a>2638: 
        A()
    else:
       A()
def cgj(): 
        
    global x
    global m
    global a
    global s
    
    if a<890: 
        ff()
    elif x>1546: 
        nx()
    elif m<1898: 
        zcv()
    else:
       kfs()
def srz(): 
        
    global x
    global m
    global a
    global s
    
    if a>1345: 
        A()
    else:
       A()
def pn(): 
        
    global x
    global m
    global a
    global s
    
    if s<1253: 
        A()
    else:
       R()
def ktx(): 
        
    global x
    global m
    global a
    global s
    
    if a<1448: 
        A()
    elif a<1571: 
        A()
    else:
       A()
def mdl(): 
        
    global x
    global m
    global a
    global s
    
    if s<786: 
        R()
    elif m<811: 
        A()
    else:
       A()
def jqq(): 
        
    global x
    global m
    global a
    global s
    
    if s>3703: 
        gjp()
    else:
       hcq()
def rjk(): 
        
    global x
    global m
    global a
    global s
    
    if s>459: 
        A()
    elif x>3536: 
        A()
    else:
       A()
def cxs(): 
        
    global x
    global m
    global a
    global s
    
    if s>846: 
        pnb()
    elif s>638: 
        R()
    else:
       A()
def jtt(): 
        
    global x
    global m
    global a
    global s
    
    if s>281: 
        gmh()
    elif x<928: 
        fnd()
    elif s<175: 
        A()
    else:
       A()
def ngq(): 
        
    global x
    global m
    global a
    global s
    
    if s>979: 
        db()
    elif x<2798: 
        tz()
    elif m>2572: 
        R()
    else:
       sqt()
def qb(): 
        
    global x
    global m
    global a
    global s
    
    if m>669: 
        R()
    else:
       A()
def pvj(): 
        
    global x
    global m
    global a
    global s
    
    if m>712: 
        R()
    elif x>690: 
        R()
    elif a>1577: 
        R()
    else:
       A()

In [270]:
global accepted

def A():
    print("A")
    global accepted
    accepted = True

def R():
    print("R")
    global accepted
    accepted = False
    
def score(d):
    global accepted
    accepted = None
    
    global x
    x = d["x"]
    global m
    m = d["m"]
    global a
    a = d["a"]
    global s
    s = d["s"]
    
    in12983y1o8uf()
    
    print(accepted)

    if accepted:
        return sum(d.values())
    else:
        return 0

In [271]:
brr = brr.replace("x", "'x'").replace("m", "'m'").replace("a", "'a'").replace("s", "'s'").replace("=",":")
brr = "\n".join(f"res += score({row})" for row in brr.split("\n"))

In [272]:
print(brr)

res += score({'x':396,'m':612,'a':2012,'s':177})
res += score({'x':39,'m':25,'a':812,'s':272})
res += score({'x':1930,'m':15,'a':99,'s':905})
res += score({'x':373,'m':1162,'a':3153,'s':1794})
res += score({'x':1045,'m':108,'a':1143,'s':639})
res += score({'x':2178,'m':621,'a':580,'s':172})
res += score({'x':1292,'m':1910,'a':122,'s':2528})
res += score({'x':275,'m':192,'a':83,'s':2992})
res += score({'x':2218,'m':157,'a':99,'s':545})
res += score({'x':822,'m':2795,'a':82,'s':150})
res += score({'x':2178,'m':12,'a':1672,'s':1504})
res += score({'x':2463,'m':620,'a':882,'s':1746})
res += score({'x':1025,'m':1370,'a':21,'s':45})
res += score({'x':3161,'m':944,'a':884,'s':1590})
res += score({'x':285,'m':849,'a':595,'s':2630})
res += score({'x':423,'m':480,'a':177,'s':2385})
res += score({'x':2022,'m':777,'a':1166,'s':371})
res += score({'x':195,'m':1446,'a':1622,'s':727})
res += score({'x':1831,'m':1163,'a':710,'s':98})
res += score({'x':2054,'m':767,'a':416,'s':462})
res += score({'x':8

In [273]:
res = 0

In [275]:
res += score({'x':396,'m':612,'a':2012,'s':177})
res += score({'x':39,'m':25,'a':812,'s':272})
res += score({'x':1930,'m':15,'a':99,'s':905})
res += score({'x':373,'m':1162,'a':3153,'s':1794})
res += score({'x':1045,'m':108,'a':1143,'s':639})
res += score({'x':2178,'m':621,'a':580,'s':172})
res += score({'x':1292,'m':1910,'a':122,'s':2528})
res += score({'x':275,'m':192,'a':83,'s':2992})
res += score({'x':2218,'m':157,'a':99,'s':545})
res += score({'x':822,'m':2795,'a':82,'s':150})
res += score({'x':2178,'m':12,'a':1672,'s':1504})
res += score({'x':2463,'m':620,'a':882,'s':1746})
res += score({'x':1025,'m':1370,'a':21,'s':45})
res += score({'x':3161,'m':944,'a':884,'s':1590})
res += score({'x':285,'m':849,'a':595,'s':2630})
res += score({'x':423,'m':480,'a':177,'s':2385})
res += score({'x':2022,'m':777,'a':1166,'s':371})
res += score({'x':195,'m':1446,'a':1622,'s':727})
res += score({'x':1831,'m':1163,'a':710,'s':98})
res += score({'x':2054,'m':767,'a':416,'s':462})
res += score({'x':814,'m':1038,'a':1871,'s':118})
res += score({'x':2109,'m':136,'a':69,'s':265})
res += score({'x':30,'m':745,'a':2265,'s':2135})
res += score({'x':92,'m':1923,'a':1269,'s':62})
res += score({'x':2952,'m':875,'a':82,'s':199})
res += score({'x':1527,'m':550,'a':926,'s':34})
res += score({'x':84,'m':1894,'a':1418,'s':3020})
res += score({'x':1419,'m':2050,'a':2634,'s':992})
res += score({'x':19,'m':1722,'a':207,'s':2292})
res += score({'x':122,'m':3195,'a':179,'s':979})
res += score({'x':970,'m':3603,'a':193,'s':1431})
res += score({'x':515,'m':681,'a':501,'s':459})
res += score({'x':682,'m':2621,'a':1754,'s':649})
res += score({'x':1972,'m':2722,'a':215,'s':2720})
res += score({'x':358,'m':2273,'a':248,'s':1091})
res += score({'x':103,'m':468,'a':48,'s':467})
res += score({'x':111,'m':251,'a':199,'s':55})
res += score({'x':1536,'m':541,'a':1850,'s':1349})
res += score({'x':318,'m':1491,'a':1124,'s':1821})
res += score({'x':2441,'m':1571,'a':981,'s':1207})
res += score({'x':639,'m':2165,'a':588,'s':658})
res += score({'x':704,'m':1423,'a':2397,'s':1207})
res += score({'x':51,'m':161,'a':833,'s':1036})
res += score({'x':375,'m':188,'a':1668,'s':681})
res += score({'x':141,'m':744,'a':732,'s':77})
res += score({'x':341,'m':1182,'a':1040,'s':249})
res += score({'x':265,'m':873,'a':281,'s':1691})
res += score({'x':1022,'m':37,'a':1865,'s':338})
res += score({'x':1580,'m':137,'a':283,'s':1031})
res += score({'x':1361,'m':1432,'a':1179,'s':2447})
res += score({'x':617,'m':1360,'a':305,'s':54})
res += score({'x':235,'m':733,'a':60,'s':2192})
res += score({'x':362,'m':181,'a':1297,'s':835})
res += score({'x':133,'m':132,'a':1327,'s':64})
res += score({'x':573,'m':263,'a':741,'s':98})
res += score({'x':799,'m':898,'a':2180,'s':732})
res += score({'x':186,'m':517,'a':681,'s':1168})
res += score({'x':47,'m':2806,'a':3577,'s':578})
res += score({'x':3218,'m':1130,'a':1204,'s':414})
res += score({'x':2468,'m':2891,'a':900,'s':2094})
res += score({'x':215,'m':1069,'a':1689,'s':1228})
res += score({'x':372,'m':937,'a':725,'s':412})
res += score({'x':1434,'m':1549,'a':1539,'s':1007})
res += score({'x':3222,'m':841,'a':1854,'s':1272})
res += score({'x':2326,'m':1637,'a':1918,'s':30})
res += score({'x':1248,'m':266,'a':750,'s':2521})
res += score({'x':110,'m':1288,'a':364,'s':749})
res += score({'x':752,'m':803,'a':257,'s':316})
res += score({'x':1777,'m':1615,'a':942,'s':110})
res += score({'x':227,'m':1524,'a':1800,'s':309})
res += score({'x':105,'m':246,'a':1723,'s':3455})
res += score({'x':713,'m':1172,'a':833,'s':804})
res += score({'x':3026,'m':39,'a':376,'s':71})
res += score({'x':946,'m':3437,'a':1461,'s':149})
res += score({'x':1878,'m':303,'a':207,'s':1081})
res += score({'x':2982,'m':2510,'a':14,'s':3078})
res += score({'x':818,'m':1042,'a':943,'s':958})
res += score({'x':2349,'m':7,'a':802,'s':822})
res += score({'x':140,'m':473,'a':2454,'s':3331})
res += score({'x':1014,'m':796,'a':1453,'s':348})
res += score({'x':1578,'m':2615,'a':539,'s':294})
res += score({'x':993,'m':276,'a':564,'s':2618})
res += score({'x':2908,'m':327,'a':705,'s':1430})
res += score({'x':46,'m':361,'a':291,'s':1076})
res += score({'x':2267,'m':212,'a':952,'s':365})
res += score({'x':677,'m':2277,'a':550,'s':1061})
res += score({'x':226,'m':1391,'a':653,'s':2321})
res += score({'x':1654,'m':15,'a':27,'s':421})
res += score({'x':974,'m':241,'a':255,'s':283})
res += score({'x':202,'m':84,'a':1805,'s':81})
res += score({'x':337,'m':529,'a':254,'s':42})
res += score({'x':765,'m':2055,'a':473,'s':413})
res += score({'x':1731,'m':81,'a':370,'s':133})
res += score({'x':2839,'m':739,'a':513,'s':990})
res += score({'x':376,'m':660,'a':924,'s':65})
res += score({'x':627,'m':1486,'a':1422,'s':2326})
res += score({'x':2688,'m':2,'a':659,'s':347})
res += score({'x':1388,'m':1650,'a':1875,'s':363})
res += score({'x':305,'m':1406,'a':1285,'s':50})
res += score({'x':1095,'m':382,'a':2098,'s':1120})
res += score({'x':1172,'m':402,'a':1935,'s':1646})
res += score({'x':27,'m':357,'a':714,'s':355})
res += score({'x':42,'m':45,'a':217,'s':1134})
res += score({'x':22,'m':415,'a':624,'s':358})
res += score({'x':324,'m':3057,'a':221,'s':3288})
res += score({'x':540,'m':179,'a':1033,'s':568})
res += score({'x':7,'m':999,'a':453,'s':316})
res += score({'x':231,'m':428,'a':1092,'s':8})
res += score({'x':2221,'m':3036,'a':22,'s':735})
res += score({'x':247,'m':516,'a':339,'s':1756})
res += score({'x':3140,'m':2229,'a':1643,'s':2012})
res += score({'x':1248,'m':500,'a':624,'s':779})
res += score({'x':1245,'m':1806,'a':579,'s':2269})
res += score({'x':989,'m':442,'a':671,'s':419})
res += score({'x':266,'m':1547,'a':992,'s':664})
res += score({'x':1078,'m':24,'a':1069,'s':1848})
res += score({'x':908,'m':1979,'a':1027,'s':956})
res += score({'x':78,'m':719,'a':1328,'s':105})
res += score({'x':901,'m':601,'a':3503,'s':1239})
res += score({'x':949,'m':851,'a':356,'s':1086})
res += score({'x':315,'m':565,'a':1868,'s':857})
res += score({'x':1426,'m':1398,'a':918,'s':1629})
res += score({'x':1286,'m':2229,'a':500,'s':16})
res += score({'x':360,'m':481,'a':2006,'s':991})
res += score({'x':1514,'m':2442,'a':1921,'s':183})
res += score({'x':2155,'m':1924,'a':616,'s':986})
res += score({'x':2121,'m':1323,'a':352,'s':958})
res += score({'x':756,'m':2750,'a':543,'s':2608})
res += score({'x':833,'m':168,'a':2410,'s':232})
res += score({'x':3,'m':810,'a':451,'s':40})
res += score({'x':1151,'m':452,'a':1642,'s':965})
res += score({'x':560,'m':2062,'a':2265,'s':846})
res += score({'x':1,'m':246,'a':905,'s':323})
res += score({'x':1642,'m':193,'a':448,'s':25})
res += score({'x':86,'m':486,'a':588,'s':75})
res += score({'x':2360,'m':5,'a':417,'s':2689})
res += score({'x':676,'m':947,'a':402,'s':355})
res += score({'x':1869,'m':44,'a':944,'s':1139})
res += score({'x':36,'m':1033,'a':588,'s':391})
res += score({'x':63,'m':31,'a':608,'s':817})
res += score({'x':1115,'m':2098,'a':617,'s':702})
res += score({'x':2117,'m':848,'a':2371,'s':3170})
res += score({'x':270,'m':2617,'a':393,'s':330})
res += score({'x':325,'m':3377,'a':870,'s':1077})
res += score({'x':275,'m':139,'a':876,'s':648})
res += score({'x':657,'m':231,'a':2197,'s':915})
res += score({'x':1934,'m':1410,'a':1965,'s':1574})
res += score({'x':251,'m':841,'a':480,'s':24})
res += score({'x':771,'m':479,'a':107,'s':2259})
res += score({'x':1364,'m':166,'a':27,'s':664})
res += score({'x':774,'m':303,'a':909,'s':93})
res += score({'x':264,'m':164,'a':3471,'s':176})
res += score({'x':681,'m':2247,'a':1660,'s':837})
res += score({'x':1348,'m':204,'a':990,'s':219})
res += score({'x':650,'m':1469,'a':1269,'s':1160})
res += score({'x':1121,'m':534,'a':2270,'s':927})
res += score({'x':322,'m':1871,'a':693,'s':170})
res += score({'x':540,'m':462,'a':929,'s':99})
res += score({'x':3015,'m':915,'a':808,'s':3661})
res += score({'x':248,'m':383,'a':471,'s':106})
res += score({'x':1536,'m':1350,'a':2067,'s':804})
res += score({'x':1656,'m':56,'a':1629,'s':995})
res += score({'x':17,'m':236,'a':288,'s':218})
res += score({'x':663,'m':1611,'a':317,'s':1272})
res += score({'x':630,'m':2397,'a':1538,'s':805})
res += score({'x':211,'m':2,'a':137,'s':514})
res += score({'x':2340,'m':191,'a':173,'s':618})
res += score({'x':2154,'m':6,'a':42,'s':884})
res += score({'x':526,'m':1292,'a':694,'s':1722})
res += score({'x':606,'m':355,'a':1532,'s':1351})
res += score({'x':1405,'m':543,'a':126,'s':3})
res += score({'x':548,'m':429,'a':1382,'s':572})
res += score({'x':95,'m':1008,'a':1790,'s':42})
res += score({'x':1796,'m':1272,'a':188,'s':201})
res += score({'x':1437,'m':3161,'a':754,'s':4})
res += score({'x':891,'m':447,'a':1389,'s':145})
res += score({'x':183,'m':894,'a':821,'s':1455})
res += score({'x':457,'m':79,'a':1586,'s':1179})
res += score({'x':1631,'m':828,'a':2697,'s':73})
res += score({'x':56,'m':506,'a':1988,'s':448})
res += score({'x':3408,'m':971,'a':2130,'s':1899})
res += score({'x':347,'m':1915,'a':193,'s':1460})
res += score({'x':308,'m':3,'a':1189,'s':191})
res += score({'x':2,'m':354,'a':2761,'s':287})
res += score({'x':634,'m':864,'a':101,'s':233})
res += score({'x':632,'m':91,'a':1935,'s':202})
res += score({'x':27,'m':1364,'a':2765,'s':2})
res += score({'x':367,'m':510,'a':473,'s':132})
res += score({'x':987,'m':2516,'a':1834,'s':1121})
res += score({'x':626,'m':845,'a':176,'s':265})
res += score({'x':577,'m':823,'a':2129,'s':23})
res += score({'x':802,'m':295,'a':389,'s':226})
res += score({'x':844,'m':301,'a':549,'s':126})
res += score({'x':1139,'m':235,'a':9,'s':1183})
res += score({'x':663,'m':2444,'a':554,'s':1516})
res += score({'x':1244,'m':355,'a':252,'s':1023})
res += score({'x':151,'m':2238,'a':392,'s':93})
res += score({'x':99,'m':1911,'a':362,'s':1272})
res += score({'x':71,'m':146,'a':46,'s':1359})
res += score({'x':1140,'m':650,'a':75,'s':1458})

R
False
R
False
R
False
A
True
A
True
R
False
R
False
R
False
R
False
R
False
A
True
A
True
A
True
R
False
A
True
A
True
R
False
A
True
A
True
R
False
A
True
R
False
A
True
R
False
R
False
R
False
A
True
A
True
R
False
R
False
R
False
A
True
R
False
R
False
R
False
A
True
A
True
R
False
R
False
A
True
R
False
A
True
R
False
A
True
R
False
A
True
A
True
R
False
A
True
A
True
R
False
A
True
A
True
A
True
R
False
R
False
R
False
A
True
A
True
A
True
A
True
R
False
A
True
A
True
A
True
R
False
A
True
A
True
A
True
A
True
R
False
A
True
R
False
A
True
R
False
A
True
R
False
R
False
R
False
A
True
A
True
R
False
R
False
A
True
R
False
A
True
A
True
R
False
A
True
R
False
A
True
A
True
A
True
R
False
R
False
A
True
R
False
A
True
R
False
R
False
R
False
R
False
A
True
R
False
A
True
A
True
A
True
A
True
A
True
R
False
R
False
R
False
R
False
R
False
A
True
R
False
R
False
R
False
R
False
R
False
A
True
A
True
R
False
A
True
A
True
A
True
R
False
R
False
R
False
A
True
A
True
A
True
R
False
A


In [276]:
res

397061